Following https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

# Define the Problem
## Given
- A set of automobile crash records
- Louisiana in 2019
- 160,186 crashes
- 76 fields per crash

## Find
- Factors that correlate with lethality

# Gather the Data
- Already have it.

# Prepare Data for Consumption


## Import Libraries

In [1]:
import sys
print ('Python version: {}'.format(sys.version))
import pandas as pd
print ('pandas version: {}'.format(pd.__version__))
import matplotlib
print ('matplotlib version: {}'.format(matplotlib.__version__))
import numpy as np
print ('NumPy version: {}'.format(np.__version__))
import scipy as sp
print ('SciPy version: {}'.format(sp.__version__))
import IPython
print ('IPython version: {}'.format(IPython.__version__))
import sklearn
print ('scikit-learn version: {}'.format(sklearn.__version__))
import random
import time
import warnings
warnings.filterwarnings('ignore')
print ('-*'*20)
from subprocess import check_output
print (check_output(['ls', './data']).decode('utf8'))

Python version: 3.9.1 (v3.9.1:1e5d33e9b9, Dec  7 2020, 12:10:52) 
[Clang 6.0 (clang-600.0.57)]
pandas version: 1.2.3
matplotlib version: 3.3.3
NumPy version: 1.19.5
SciPy version: 1.6.0
IPython version: 7.21.0
scikit-learn version: 0.24.1
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
2019_Crash_1_Database.csv
CODE_TB.csv



## Load Data Modeling Libraries

In [2]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
#from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection, model_selection, metrics
from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## Explore Data

In [3]:
data = pd.read_csv('./data/2019_Crash_1_Database.csv', parse_dates = ['crash_date', 'crash_hour', 'crash_time'])

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160186 entries, 0 to 160185
Data columns (total 76 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   route                160186 non-null  object        
 1   milepoint            100414 non-null  float64       
 2   num_tot_kil          160186 non-null  int64         
 3   num_tot_inj          160186 non-null  int64         
 4   crash_date           160186 non-null  datetime64[ns]
 5   f_harm_ev_cd1        160186 non-null  object        
 6   m_harm_ev_cd1        160186 non-null  object        
 7   man_coll_cd          160186 non-null  object        
 8   crash_type           160186 non-null  object        
 9   surf_cond_cd         160186 non-null  object        
 10  crash_num            160186 non-null  object        
 11  parish_cd            160186 non-null  int64         
 12  crash_hour           160186 non-null  object        
 13  intersection  

In [5]:
data.describe()

,milepoint,num_tot_kil,num_tot_inj,parish_cd,intersection,crash_year,logmile,lrs_logmile,adt,alcohol,...,ORIG_LONGITUDE,DOTD_LATITUDE,DOTD_LONGITUDE,parish_cd.1,pri_hwy_num,milepost,pri_dist,dr_age_1,dr_age_2,num_veh
count,100414.000000,160186.000000,160186.000000,160186.000000,160186.000000,160186.0,100672.000000,158755.000000,100899.000000,160186.000000,...,160186.000000,159868.000000,159869.000000,160186.000000,160186.000000,160186.000000,160186.000000,160185.000000,134521.000000,160186.000000
mean,66.614763,0.004538,0.452586,28.480598,0.367329,2019.0,5.352676,3.600491,36328.367526,0.029066,...,67.185435,30.512016,-91.098355,28.519465,214.757894,28.246646,77.302522,57.584056,50.227912,1.915149
std,85.264620,0.072229,0.973207,15.426490,0.482079,0.0,5.736082,4.959737,39261.577426,0.167993,...,44.389833,2.049432,5.648987,15.430538,637.046823,77.795943,441.380509,54.058061,39.525076,0.514440
min,-0.580000,0.000000,0.000000,1.000000,0.000000,2019.0,-9.999000,0.000000,0.000000,0.000000,...,-95.012300,0.000000,-94.113000,0.000000,0.000000,-2.000000,-2.000000,2.000000,2.000000,1.000000
25%,4.197000,0.000000,0.000000,17.000000,0.000000,2019.0,1.504000,0.428000,10400.000000,0.000000,...,89.750213,30.002798,-92.199576,17.000000,0.000000,0.000000,0.000000,25.000000,29.000000,2.000000
50%,20.763500,0.000000,0.000000,28.000000,0.000000,2019.0,3.602000,1.680000,23100.000000,0.000000,...,90.920592,30.295656,-91.117383,28.000000,10.000000,0.000000,0.500000,38.000000,40.000000,2.000000
75%,110.205000,0.000000,1.000000,37.000000,1.000000,2019.0,7.276000,4.830000,47400.000000,0.000000,...,92.033123,30.561540,-90.185287,37.000000,90.000000,10.900000,50.000000,60.000000,57.000000,2.000000
max,430.902000,4.000000,64.000000,64.000000,1.000000,2019.0,60.230000,60.230000,208660.000000,1.000000,...,99.677620,33.019790,0.000000,64.000000,11382.000000,999.990000,99999.990000,200.000000,200.000000,16.000000


In [6]:
data.describe(include=['O'])

,route,f_harm_ev_cd1,m_harm_ev_cd1,man_coll_cd,crash_type,surf_cond_cd,crash_num,crash_hour,invest_agency_cd,travel_dirs,...,veh_lighting_1,veh_lighting_2,traff_cntl_cond_1,traff_cntl_cond_2,pri_road_dir,lighting_cd,dr_cond_cd1,dr_cond_cd2,veh_cond_cd1,veh_cond_cd2
count,160186,160186,160186,160186,160186,160186,160186,160186,160186,160186,...,160185,134521,160185,134521,160186,160186,160186,160186,160186,160186
unique,1024,45,45,12,19,10,160186,25,4,199,...,6,5,25,25,9,11,14,14,14,14
top,0,S,S,B,S,A,201910281701013776-,17,B,EE,...,A,B,R,R,,A,B,A,K,K
freq,59297,125542,123871,55470,76102,134989,1,13943,96622,21023,...,50820,43117,45561,44720,131708,116211,79780,120975,133539,127910


In [7]:
data.sample(10)

,route,milepoint,num_tot_kil,num_tot_inj,crash_date,f_harm_ev_cd1,m_harm_ev_cd1,man_coll_cd,crash_type,surf_cond_cd,...,traff_cntl_cond_1,traff_cntl_cond_2,pri_road_dir,lighting_cd,num_veh,crash_time,dr_cond_cd1,dr_cond_cd2,veh_cond_cd1,veh_cond_cd2
95671,0,NaN,0,0,2019-08-09,S,X,E,H,A,...,A,NaN,,D,1,2021-04-16 20:27:00,A,,K,
128458,I020,103.837,0,0,2019-10-21,AA,AA,A,H,B,...,R,NaN,,A,1,2021-04-16 07:10:00,B,,K,
19464,0,NaN,0,0,2019-02-17,S,S,Z,S,A,...,R,R,,A,2,2021-04-16 08:38:00,Y,A,K,K
92173,3025S,0.414,0,0,2019-07-31,S,S,G,S,A,...,Z,Z,,A,2,2021-04-16 16:24:00,B,A,K,K
67423,0,NaN,0,1,2019-05-31,S,S,B,S,A,...,C,C,,A,2,2021-04-16 07:22:00,B,A,K,K
104569,80,17.060,0,3,2019-08-29,S,S,D,S,A,...,C,V,,A,2,2021-04-16 15:21:00,B,A,K,K
143679,1031,0.068,0,0,2019-11-22,S,S,B,R,B,...,C,C,,D,3,2021-04-16 17:56:00,B,A,K,K
82529,8,77.774,0,0,2019-07-07,L,S,K,S,A,...,S,S,,D,2,2021-04-16 21:26:00,A,A,K,K
156975,I020,101.435,0,0,2019-12-22,J,S,B,S,B,...,R,V,,B,2,2021-04-16 02:10:00,B,Z,K,K
17229,0,NaN,0,0,2019-02-12,S,S,B,U,A,...,C,C,,A,2,2021-04-16 17:43:00,C,A,K,K


### Columns with null values

In [8]:
pd.set_option('max_rows', None)
data.isnull().sum()

route                       0
milepoint               59772
num_tot_kil                 0
num_tot_inj                 0
crash_date                  0
f_harm_ev_cd1               0
m_harm_ev_cd1               0
man_coll_cd                 0
crash_type                  0
surf_cond_cd                0
crash_num                   0
parish_cd                   0
crash_hour                  0
intersection                0
invest_agency_cd            0
travel_dirs                 0
prior_movements             0
crash_year                  0
csect                       0
logmile                 59514
lrs_id                      0
lrs_logmile              1431
adt                     59287
alcohol                     0
veh_type_cd1                0
veh_type_cd2                0
quadrant                    0
spotted_by                  0
intersection_id         94852
severity_cd                 0
city_cd                     0
roadway_departure           0
lane_departure              0
road_rel_c

In [9]:
pd.reset_option('max_rows')

- It appears that 20,509 records are single-vehicle accidents, because there are exactly 20,509 blanks in the "vehicle 2" columns. 
- Similarly, there are 126,876 records that don't involve a pedestrian.

## Clean Data

In [10]:
def Analysis(s):
    print ('Analysis of field ', s)
    n = len(pd.unique(data[s]))
    print (n, "\t Number of unique values in ", s)
    print (data[s].isnull().sum(), "\t Number of null values in ", s)
    print ()
    print ("\t Value Counts")
    print (data[s].value_counts())
    print ()
    if n<100:
        print ("\t Truncated list of unique values in ", s, ": ")
        if data[s].dtype in ['int8','float64']:
            print (sorted(pd.unique(data[s])))
        else:
            print (pd.unique(data[s]))
    print ('-*'*20)
    print ()

In [11]:
def Make_Category(col):
    data[col] = data[col].astype('category')
    new_col = col + '_codes'
    data[new_col] = data[col].cat.codes
    Analysis(new_col)

### 'route'

In [12]:
Analysis('route')

Analysis of field  route
1024 	 Number of unique values in  route
0 	 Number of null values in  route

	 Value Counts
0        59297
I010     11087
90        4729
61        4137
190       3717
         ...  
3116         1
3076         1
990          1
607          1
3248         1
Name: route, Length: 1024, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Too many unique values that don't correlate to anything we can see.  It may be true that windy country roads, or roads with high speed limits, or roads with lots of drag racing, might have higher fatality rates, but we don't have that information to correlate with the route name.  

In [13]:
data.drop(['route'], axis=1, inplace=True)

### 'milepoint'

In [14]:
Analysis ('milepoint')

Analysis of field  milepoint
53808 	 Number of unique values in  milepoint
59772 	 Number of null values in  milepoint

	 Value Counts
0.000      326
194.280     45
3.851       38
6.087       36
257.836     36
          ... 
13.129       1
67.177       1
74.721       1
189.609      1
35.843       1
Name: milepoint, Length: 53807, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I can't see any predictive value here.  

In [15]:
data.drop(['milepoint'], axis=1, inplace=True)

### 'num_tot_kil'

In [16]:
Analysis ('num_tot_kil')

Analysis of field  num_tot_kil
5 	 Number of unique values in  num_tot_kil
0 	 Number of null values in  num_tot_kil

	 Value Counts
0    159505
1       644
2        29
3         7
4         1
Name: num_tot_kil, dtype: int64

	 Truncated list of unique values in  num_tot_kil : 
[0 1 3 2 4]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



We're only interested in whether the crash was fatal, whether num_tot_kil > 0.  Collapse 2, 3, and 4 into 1.

In [17]:
mapping = {0:0, 1:1, 2:1, 3:1, 4:1}
data['fatal'] = data['num_tot_kil'].map(mapping).astype(int)
Analysis('fatal')
data.sample(10)

Analysis of field  fatal
2 	 Number of unique values in  fatal
0 	 Number of null values in  fatal

	 Value Counts
0    159505
1       681
Name: fatal, dtype: int64

	 Truncated list of unique values in  fatal : 
[0 1]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



,num_tot_kil,num_tot_inj,crash_date,f_harm_ev_cd1,m_harm_ev_cd1,man_coll_cd,crash_type,surf_cond_cd,crash_num,parish_cd,...,traff_cntl_cond_2,pri_road_dir,lighting_cd,num_veh,crash_time,dr_cond_cd1,dr_cond_cd2,veh_cond_cd1,veh_cond_cd2,fatal
30431,0,0,2019-03-12,K,GG,A,H,A,190312120516536-,61,...,NaN,,A,1,2021-04-16 09:46:00,B,,K,,0
18031,0,1,2019-02-13,S,S,J,S,A,201902151701001901-,17,...,R,W,A,2,2021-04-16 13:37:00,B,A,K,K,0
149519,0,1,2019-12-06,S,S,B,S,A,20190042899-,52,...,S,,B,2,2021-04-16 17:10:00,C,A,K,K,0
47150,0,0,2019-04-16,S,S,B,S,A,201904241701005339-,17,...,E,,A,2,2021-04-16 14:33:00,B,A,K,K,0
99769,0,0,2019-08-19,S,S,F,S,B,190820074637074-,23,...,S,S,A,2,2021-04-16 11:55:00,B,A,K,K,0
43208,0,0,2019-04-08,S,S,F,R,A,190410210702100-,10,...,E,E,A,3,2021-04-16 16:41:00,B,A,K,K,0
45687,0,0,2019-04-13,T,T,Z,E,A,190414081049189-,26,...,V,,C,2,2021-04-16 20:10:00,B,,K,K,0
113383,0,3,2019-09-18,S,S,C,R,B,190918132550549-,8,...,Q,,A,3,2021-04-16 14:38:00,B,A,K,K,0
88934,0,0,2019-07-23,HH,HH,A,Q,B,190730024628178-,36,...,R,,B,2,2021-04-16 03:45:00,C,A,Z,K,0
66906,0,0,2019-05-30,S,S,A,Q,A,201905311701007001-,17,...,V,,C,2,2021-04-16 23:27:00,Y,Z,K,K,0


### 'num_tot_inj'

In [18]:
Analysis('num_tot_inj')

Analysis of field  num_tot_inj
27 	 Number of unique values in  num_tot_inj
0 	 Number of null values in  num_tot_inj

	 Value Counts
0     115335
1      28915
2       9638
3       3596
4       1477
5        654
6        285
7        137
8         64
9         27
10        18
11        12
12         6
19         3
13         3
15         2
16         2
20         2
52         2
17         1
18         1
14         1
22         1
23         1
36         1
37         1
64         1
Name: num_tot_inj, dtype: int64

	 Truncated list of unique values in  num_tot_inj : 
[ 0  1  3  2  4  7  6  8  5 16  9 22 23 10 13 19 12 11 18 37 36 20 52 15
 17 14 64]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Not sure what we'll do with this information.  It might be useful.  

### crash_date

In [19]:
Analysis('crash_date')

Analysis of field  crash_date
365 	 Number of unique values in  crash_date
0 	 Number of null values in  crash_date

	 Value Counts
2019-10-25    788
2019-04-04    699
2019-03-01    661
2019-12-10    643
2019-05-10    637
             ... 
2019-01-27    262
2019-05-27    261
2019-11-28    236
2019-07-13    221
2019-12-25    211
Name: crash_date, Length: 365, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Two things might be useful here.  One is the day of the week, and the other is the month.  

dayofweek gives 0=Monday and 6=Sunday

In [20]:
data['crash_dayofweek'] = data['crash_date'].dt.dayofweek
Analysis ('crash_dayofweek')

Analysis of field  crash_dayofweek
7 	 Number of unique values in  crash_dayofweek
0 	 Number of null values in  crash_dayofweek

	 Value Counts
4    27230
3    25079
2    24449
1    24229
0    23411
5    19729
6    16059
Name: crash_dayofweek, dtype: int64

	 Truncated list of unique values in  crash_dayofweek : 
[1 2 3 4 5 6 0]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [21]:
data['crash_month'] = data['crash_date'].dt.month
Analysis ('crash_month')

Analysis of field  crash_month
12 	 Number of unique values in  crash_month
0 	 Number of null values in  crash_month

	 Value Counts
10    14820
3     14443
5     14382
4     13818
8     13475
12    13326
11    13240
9     13053
1     12762
2     12342
7     12317
6     12208
Name: crash_month, dtype: int64

	 Truncated list of unique values in  crash_month : 
[ 1  2  3  4  5  6  7  8  9 10 11 12]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### 'f_harm_ev_cd1'

The values in this category give the (primary?) description of how the vehicle left the road.  

It comes in five flavors, although this database only has the "First" and "Most." 

I presume the '1' refers to 'vehicle 1.'

| Code | Meaning |
|---|---|
| f_harm_ev_cd | First Harmful Event Code |
| s_harm_ev_cd | Second Harmful Event Code |
| t_harm_ev_cd | Third Harmful Event Code |
| fo_harm_ev_cd | Fourth Harmful Event Code |
| m_harm_ev_cd | Most Harmful Event Code |



| Code | Meaning
| -- | ---------------------------------------------------------------------------- |
| A  | OVERTURN/ROLLOVER                                                            |
| AA | BRIDGE RAIL                                                                  |
| B  | FIRE/EXPLOSION                                                               |
| BB | CULVERT                                                                      |
| C  | IMMERSION                                                                    |
| CC | CURB                                                                         |
| D  | JACKKNIFE                                                                    |
| DD | DITCH                                                                        |
| E  | CARGO/EQUIPMENT LOSS OR SHIFT                                                |
| EE | EMBANKMENT                                                                   |
| F  | FELL/JUMPED FROM MOTOR VEHICLE                                               |
| FF | GUARDRAIL FACE                                                               |
| G  | THROWN OR FALLING OBJECT                                                     |
| GG | GUARDRAIL END                                                                |
| H  | EQUIPMENT FAILURE (BLOWN TIRE, BRAKE FAILURE, ETC.)                          |
| HH | CONCRETE TRAFFIC SUPPORT                                                     |
| I  | SEPARATION OF UNITS IN TRANSPORT                                             |
| II | OTHER TRAFFIC BARRIER                                                        |
| J  | RAN OFF ROAD RIGHT                                                           |
| JJ | TREE (STANDING)                                                              |
| K  | RAN OFF ROAD LEFT                                                            |
| KK | UTILITY POLE/LIGHT SUPPORT                                                   |
| L  | CROSSED MEDIAN/CENTERLINE                                                    |
| LL | TRAFFIC SIGN SUPPORT                                                         |
| M  | DOWNHILL RUNAWAY                                                             |
| MM | TRAFFIC SIGNAL SUPPORT                                                       |
| N  | OTHER NON-COLLISION                                                          |
| NN | OTHER POST, POLE, OR SUPPORT                                                 |
| O  | PEDESTRIAN                                                                   |
| OO | FENCE                                                                        |
| P  | PEDALCYCLE                                                                   |
| PP | MAILBOX                                                                      |
| Q  | RAILWAY VEHICLE (TRAIN, ENGINE)                                              |
| QQ | OTHER FIXED OBJECT (WALL, BUILDING, TUNNEL, ETC.)                            |
| R  | ANIMAL                                                                       |
| S  | MOTOR VEHICLE IN TRANSPORT                                                   |
| T  | PARKED MOTOR VEHICLE                                                         |
| U  | STRUCK BY FALLING, SHIFTING CARGO OR ANYTHING SET IN MOTION BY MOTOR VEHICLE |
| V  | WORK ZONE/MAINTENANCE EQUIPMENT                                              |
| W  | OTHER NON-FIXED OBJECT                                                       |
| X  | IMPACT ATTENUATOR/CRASH CUSHION                                              |
| Y  | BRIDGE OVERHEAD STRUCTURE                                                    |
| YY | UNKNOWN                                                                      |
| Z  | BRIDGE PIER OR SUPPORT                                                       |

In [22]:
Analysis('f_harm_ev_cd1')

Analysis of field  f_harm_ev_cd1
45 	 Number of unique values in  f_harm_ev_cd1
0 	 Number of null values in  f_harm_ev_cd1

	 Value Counts
S     125542
J       7641
L       4868
T       4200
K       3355
R       2123
YY      2109
N       1252
O        957
DD       814
W        624
CC       589
KK       575
H        538
         531
QQ       412
E        362
JJ       335
HH       327
P        324
A        313
AA       252
PP       252
LL       249
NN       243
FF       209
OO       166
II       108
G        107
BB        93
Y         80
U         78
GG        66
X         58
F         55
V         54
D         54
I         51
EE        50
MM        49
Q         46
C         28
Z         21
B         15
M         11
Name: f_harm_ev_cd1, dtype: int64

	 Truncated list of unique values in  f_harm_ev_cd1 : 
['S ' 'R ' 'K ' '  ' 'T ' 'J ' 'KK' 'YY' 'L ' 'QQ' 'N ' 'DD' 'PP' 'W '
 'NN' 'O ' 'AA' 'CC' 'HH' 'H ' 'X ' 'E ' 'LL' 'JJ' 'FF' 'BB' 'V ' 'I '
 'A ' 'MM' 'GG' 'EE' 'U ' 'OO' 'F ' 'P ' 'G

These categories can't be ordered, but let's convert them to numbers.  
Replace the blank entries with "YY", meaning 'unknown."

In [23]:
data['f_harm_ev_cd1'] = data['f_harm_ev_cd1'].replace('  ', 'YY')
Analysis('f_harm_ev_cd1')

Analysis of field  f_harm_ev_cd1
44 	 Number of unique values in  f_harm_ev_cd1
0 	 Number of null values in  f_harm_ev_cd1

	 Value Counts
S     125542
J       7641
L       4868
T       4200
K       3355
YY      2640
R       2123
N       1252
O        957
DD       814
W        624
CC       589
KK       575
H        538
QQ       412
E        362
JJ       335
HH       327
P        324
A        313
AA       252
PP       252
LL       249
NN       243
FF       209
OO       166
II       108
G        107
BB        93
Y         80
U         78
GG        66
X         58
F         55
D         54
V         54
I         51
EE        50
MM        49
Q         46
C         28
Z         21
B         15
M         11
Name: f_harm_ev_cd1, dtype: int64

	 Truncated list of unique values in  f_harm_ev_cd1 : 
['S ' 'R ' 'K ' 'YY' 'T ' 'J ' 'KK' 'L ' 'QQ' 'N ' 'DD' 'PP' 'W ' 'NN'
 'O ' 'AA' 'CC' 'HH' 'H ' 'X ' 'E ' 'LL' 'JJ' 'FF' 'BB' 'V ' 'I ' 'A '
 'MM' 'GG' 'EE' 'U ' 'OO' 'F ' 'P ' 'G ' 'Q ' 'II' 'Y ' 

In [24]:
Make_Category('f_harm_ev_cd1')

Analysis of field  f_harm_ev_cd1_codes
44 	 Number of unique values in  f_harm_ev_cd1_codes
0 	 Number of null values in  f_harm_ev_cd1_codes

	 Value Counts
35    125542
18      7641
22      4868
36      4200
20      3355
42      2640
34      2123
26      1252
28       957
7        814
39       624
5        589
21       575
14       538
33       412
8        362
19       335
15       327
30       324
0        313
1        252
31       252
23       249
27       243
11       209
29       166
17       108
12       107
3         93
41        80
37        78
13        66
40        58
10        55
6         54
38        54
16        51
9         50
25        49
32        46
4         28
43        21
2         15
24        11
Name: f_harm_ev_cd1_codes, dtype: int64

	 Truncated list of unique values in  f_harm_ev_cd1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
-

### m_harm_ev_cd1
Same categories as f_harm_ev_cd1
 

In [25]:
Analysis ( 'm_harm_ev_cd1' )

Analysis of field  m_harm_ev_cd1
45 	 Number of unique values in  m_harm_ev_cd1
0 	 Number of null values in  m_harm_ev_cd1

	 Value Counts
S     123871
T       4579
        3464
DD      3423
JJ      3031
A       2415
R       2130
YY      1960
KK      1671
QQ      1072
O       1056
BB      1036
W        746
HH       696
N        647
AA       640
OO       575
J        558
LL       555
FF       541
NN       540
PP       527
EE       523
II       493
L        452
CC       434
K        335
E        331
P        330
H        212
GG       180
C        175
X        119
U        113
G        112
F         99
MM        99
Y         94
B         75
D         73
Q         64
V         57
Z         48
I         29
M          6
Name: m_harm_ev_cd1, dtype: int64

	 Truncated list of unique values in  m_harm_ev_cd1 : 
['S ' '  ' 'R ' 'DD' 'T ' 'BB' 'JJ' 'KK' 'HH' 'LL' 'YY' 'QQ' 'J ' 'L '
 'N ' 'PP' 'OO' 'AA' 'P ' 'A ' 'NN' 'C ' 'GG' 'W ' 'FF' 'H ' 'CC' 'K '
 'X ' 'II' 'E ' 'O ' 'EE' 'V ' 'U ' 'F ' 'M

Do the same thing, where we replace the blank values with 'YY' for 'Unknown.'

In [26]:
data['m_harm_ev_cd1'] = data['m_harm_ev_cd1'].replace('  ', 'YY')
Analysis('m_harm_ev_cd1')

Analysis of field  m_harm_ev_cd1
44 	 Number of unique values in  m_harm_ev_cd1
0 	 Number of null values in  m_harm_ev_cd1

	 Value Counts
S     123871
YY      5424
T       4579
DD      3423
JJ      3031
A       2415
R       2130
KK      1671
QQ      1072
O       1056
BB      1036
W        746
HH       696
N        647
AA       640
OO       575
J        558
LL       555
FF       541
NN       540
PP       527
EE       523
II       493
L        452
CC       434
K        335
E        331
P        330
H        212
GG       180
C        175
X        119
U        113
G        112
MM        99
F         99
Y         94
B         75
D         73
Q         64
V         57
Z         48
I         29
M          6
Name: m_harm_ev_cd1, dtype: int64

	 Truncated list of unique values in  m_harm_ev_cd1 : 
['S ' 'YY' 'R ' 'DD' 'T ' 'BB' 'JJ' 'KK' 'HH' 'LL' 'QQ' 'J ' 'L ' 'N '
 'PP' 'OO' 'AA' 'P ' 'A ' 'NN' 'C ' 'GG' 'W ' 'FF' 'H ' 'CC' 'K ' 'X '
 'II' 'E ' 'O ' 'EE' 'V ' 'U ' 'F ' 'MM' 'G ' 'Q ' 'D ' 

...and convert to categorical values.  

In [27]:
Make_Category('m_harm_ev_cd1')

Analysis of field  m_harm_ev_cd1_codes
44 	 Number of unique values in  m_harm_ev_cd1_codes
0 	 Number of null values in  m_harm_ev_cd1_codes

	 Value Counts
35    123871
42      5424
36      4579
7       3423
19      3031
0       2415
34      2130
21      1671
33      1072
28      1056
3       1036
39       746
15       696
26       647
1        640
29       575
18       558
23       555
11       541
27       540
31       527
9        523
17       493
22       452
5        434
20       335
8        331
30       330
14       212
13       180
4        175
40       119
37       113
12       112
10        99
25        99
41        94
2         75
6         73
32        64
38        57
43        48
16        29
24         6
Name: m_harm_ev_cd1_codes, dtype: int64

	 Truncated list of unique values in  m_harm_ev_cd1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
-

The two categories, f_harm_ev_cd1 and m_harm_ev_cd1, are interesting if they're different, but that's another topic.

### man_coll_cd

Manner of Collision Code

| Code | Meaning |
|---|---|
| A | NON-COLLISION WITH MOTOR VEHICLE|
| B | REAR END|
| C | HEAD-ON|
| D | RIGHT ANGLE|
| E | LEFT TURN - ANGLE|
| F | LEFT TURN - OPPOSITE DIRECTION|
| G | LEFT TURN - SAME DIRECTION|
| H | RIGHT TURN - SAME DIRECTION|
| I | RIGHT TURN - OPPOSITE DIRECTION|
| J | SIDESWIPE - SAME DIRECTION|
| K | SIDESWIPE - OPPOSITE DIRECTION|

In [28]:
Analysis('man_coll_cd')

Analysis of field  man_coll_cd
12 	 Number of unique values in  man_coll_cd
0 	 Number of null values in  man_coll_cd

	 Value Counts
B    55470
A    25424
J    21154
D    20348
Z    14466
F     6797
G     3871
H     3131
E     3116
K     2986
C     2537
I      886
Name: man_coll_cd, dtype: int64

	 Truncated list of unique values in  man_coll_cd : 
['D' 'C' 'A' 'B' 'E' 'Z' 'H' 'J' 'K' 'I' 'F' 'G']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No blanks, so let's just turn it into categories.  

In [29]:
Make_Category('man_coll_cd')

Analysis of field  man_coll_cd_codes
12 	 Number of unique values in  man_coll_cd_codes
0 	 Number of null values in  man_coll_cd_codes

	 Value Counts
1     55470
0     25424
9     21154
3     20348
11    14466
5      6797
6      3871
7      3131
4      3116
10     2986
2      2537
8       886
Name: man_coll_cd_codes, dtype: int64

	 Truncated list of unique values in  man_coll_cd_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### crash_type
I haven't been able to figure out what the codes signify.

In [30]:
Analysis('crash_type')

Analysis of field  crash_type
19 	 Number of unique values in  crash_type
0 	 Number of null values in  crash_type

	 Value Counts
S    76102
U    28173
R     8657
Q     8629
P     6973
E     6317
G     5602
K     3020
N     2939
T     2842
X     2408
A     1711
F     1552
D     1523
H     1315
M     1222
B      969
C      153
J       79
Name: crash_type, dtype: int64

	 Truncated list of unique values in  crash_type : 
['S' 'B' 'N' 'P' 'R' 'E' 'G' 'U' 'K' 'X' 'T' 'H' 'A' 'Q' 'M' 'F' 'D' 'C'
 'J']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [31]:
Make_Category('crash_type')

Analysis of field  crash_type_codes
19 	 Number of unique values in  crash_type_codes
0 	 Number of null values in  crash_type_codes

	 Value Counts
15    76102
17    28173
14     8657
13     8629
12     6973
4      6317
6      5602
9      3020
11     2939
16     2842
18     2408
0      1711
5      1552
3      1523
7      1315
10     1222
1       969
2       153
8        79
Name: crash_type_codes, dtype: int64

	 Truncated list of unique values in  crash_type_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### surf_cond_cd
Surface Condition Code

| Code | Meaning |
|---|---|
| A | DRY|
| B | WET|
| C | SNOW/SLUSH|
| D | ICE|
| E | CONTAMINANT (SAND, MUD, DIRT, OIL, ETC.)|
| Y | UNKNOWN|
| Z | OTHER|![image.png](attachment:image.png)

In [32]:
Analysis('surf_cond_cd')

Analysis of field  surf_cond_cd
10 	 Number of unique values in  surf_cond_cd
0 	 Number of null values in  surf_cond_cd

	 Value Counts
A    134989
B     24025
Y       681
2       195
E        79
D        77
         62
Z        36
C        25
1        17
Name: surf_cond_cd, dtype: int64

	 Truncated list of unique values in  surf_cond_cd : 
['A' 'B' 'Y' '2' 'E' 'C' ' ' '1' 'D' 'Z']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I don't know what '1' and '2' signify, and I don't see a practical difference between 'Unknown' and 'Other,' so I'm going to merge all of those, with the blanks, into one category.  

In [33]:
for x in [' ', '1', '2', 'Y']:
    data['surf_cond_cd'] = data['surf_cond_cd'].replace(x,'Z')
Analysis('surf_cond_cd')

Analysis of field  surf_cond_cd
6 	 Number of unique values in  surf_cond_cd
0 	 Number of null values in  surf_cond_cd

	 Value Counts
A    134989
B     24025
Z       991
E        79
D        77
C        25
Name: surf_cond_cd, dtype: int64

	 Truncated list of unique values in  surf_cond_cd : 
['A' 'B' 'Z' 'E' 'C' 'D']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [34]:
Make_Category('surf_cond_cd')

Analysis of field  surf_cond_cd_codes
6 	 Number of unique values in  surf_cond_cd_codes
0 	 Number of null values in  surf_cond_cd_codes

	 Value Counts
0    134989
1     24025
5       991
4        79
3        77
2        25
Name: surf_cond_cd_codes, dtype: int64

	 Truncated list of unique values in  surf_cond_cd_codes : 
[0, 1, 2, 3, 4, 5]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### crash_num

In [35]:
Analysis('crash_num')

Analysis of field  crash_num
160186 	 Number of unique values in  crash_num
0 	 Number of null values in  crash_num

	 Value Counts
201910281701013776-    1
20190016257-           1
191231053351919-       1
191026162940373-       1
20190041788-           1
                      ..
190608124806795-       1
190928083907985-       1
190913175921301-       1
190812160221441-       1
190106065455782-       1
Name: crash_num, Length: 160186, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Oh, this is useless.  

In [36]:
data.drop(['crash_num'], axis=1, inplace=True)

### parish_cd

| Code | Parish |
|---|---|
| 1 | ACADIA|	
| 2 | ALLEN|	
| 3 | ASCENSION|	
| 4 | ASSUMPTION|	
| 5 | AVOYELLES|	
| 6 | BEAUREGARD|	
| 7 | BEINVILLE|	
| 8 | BOSSIER|	
| 9 | CADDO|	
| 10 | CALCASIEU|	
| 11 | CALDWELL|	
| 12 | CAMERON|	
| 13 | CATAHOULA|	
| 14 | CLAIBORNE|	
| 15 | CONCORDIA|	
| 16 | DESOTO|	
| 17 | EAST BATON ROUGE|	
| 18 | EAST CARROLL|	
| 19 | EAST FELICIANA|	
| 20 | EVANGELINE|	
| 21 | FRANKLIN|	
| 22 | GRANT|	
| 23 | IBERIA|	
| 24 | IBERVILLE|	
| 25 | JACKSON|	
| 26 | JEFFERSON|	
| 27 | JEFFERSON DAVIS|	
| 28 | LAFAYETTE|	
| 29 | LAFOURCHE|	
| 30 | LASALLE|	
| 31 | LINCOLN|	
| 32 | LIVINGSTON|	
| 33 | MADISON|	
| 34 | MOREHOUSE|	
| 35 | NATCHITOCHES|	
| 36 | ORLEANS|	
| 37 | OUACHITA|	
| 38 | PLAQUEMINES|	
| 39 | POINT COUPEE|	
| 40 | RAPIDES|	
| 41 | RED RIVER|	
| 42 | RICHLAND|	
| 43 | SABINE|	
| 44 | ST. BERNARD|	
| 45 | ST. CHARLES|	
| 46 | ST. HELENA|	
| 47 | ST. JAMES|	
| 48 | ST. JOHN THE BAPTIST|	
| 49 | ST. LANDRY|	
| 50 | ST. MARTIN|	
| 51 | ST. MARY|	
| 52 | ST. TAMMANY|	
| 53 | TANGIPAHOA|	
| 54 | TENSAS|	
| 55 | TERREBONNE|	
| 56 | UNION|	
| 57 | VERMILLION|	
| 58 | VERNON|	
| 59 | WASHINGTON|	
| 60 | WEBSTER|	
| 61 | WEST BATON ROUGE|	
| 62 | WEST CARROLL|	
| 63 | WEST FELICIANA|	
| 64 | WINN|	![image.png](attachment:image.png)

In [37]:
Analysis('parish_cd')

Analysis of field  parish_cd
64 	 Number of unique values in  parish_cd
0 	 Number of null values in  parish_cd

	 Value Counts
17    22035
36    19963
26    16512
28    11030
9      9867
      ...  
11      142
18      112
64      108
54       96
13       76
Name: parish_cd, Length: 64, dtype: int64

	 Truncated list of unique values in  parish_cd : 
[36 17  9 28 49 43  2  1 26 20 31 32  8 55 37 33 52 10 23 50 40  3 38 53
 39 35 48 29 24 61 14 59 46 44 45 54 19 60 22  6 58 62  4  5 47 16 41 51
 56 27 18 13 11 64 15 21 57 34 42 30 25  7 63 12]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



That seems straightforward and clean.

### crash_hour

In [38]:
Analysis('crash_hour')

Analysis of field  crash_hour
25 	 Number of unique values in  crash_hour
0 	 Number of null values in  crash_hour

	 Value Counts
17    13943
16    13379
15    13081
14    10706
12    10601
18    10135
13     9842
7      8951
11     8758
8      7711
10     7213
19     6605
9      6405
20     5404
21     4662
6      4541
22     3602
5      2762
23     2682
0      2033
1      1842
2      1694
4      1582
3      1457
        595
Name: crash_hour, dtype: int64

	 Truncated list of unique values in  crash_hour : 
['17' '23' '22' '18' '19' '20' '3' '5' '16' '0' '1' '2' '14' '4' '8' '15'
 '13' '12' '10' '7' '6' '  ' '9' '21' '11']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



What to do with the blanks?  I'm going to call it '24.'

In [39]:
data['crash_hour'] = data['crash_hour'].replace('  ', '24')
Analysis ('crash_hour')

Analysis of field  crash_hour
25 	 Number of unique values in  crash_hour
0 	 Number of null values in  crash_hour

	 Value Counts
17    13943
16    13379
15    13081
14    10706
12    10601
18    10135
13     9842
7      8951
11     8758
8      7711
10     7213
19     6605
9      6405
20     5404
21     4662
6      4541
22     3602
5      2762
23     2682
0      2033
1      1842
2      1694
4      1582
3      1457
24      595
Name: crash_hour, dtype: int64

	 Truncated list of unique values in  crash_hour : 
['17' '23' '22' '18' '19' '20' '3' '5' '16' '0' '1' '2' '14' '4' '8' '15'
 '13' '12' '10' '7' '6' '24' '9' '21' '11']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### intersection
I don't know which of '0' and '1' indicates that the accident occurred at an intersection, but that's probably what this field signifies.

In [40]:
Analysis('intersection')

Analysis of field  intersection
2 	 Number of unique values in  intersection
0 	 Number of null values in  intersection

	 Value Counts
0    101345
1     58841
Name: intersection, dtype: int64

	 Truncated list of unique values in  intersection : 
[1 0]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### invest_agency_cd

| Code | Agency |
|---|---|
| A | STATE|
| B | CITY|
| C | PARISH|
| Z | OTHER|

In [41]:
Analysis('invest_agency_cd')

Analysis of field  invest_agency_cd
4 	 Number of unique values in  invest_agency_cd
0 	 Number of null values in  invest_agency_cd

	 Value Counts
B    96622
A    33834
C    28848
Z      882
Name: invest_agency_cd, dtype: int64

	 Truncated list of unique values in  invest_agency_cd : 
['B' 'C' 'A' 'Z']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Looks good.

### travel_dirs

In [42]:
Analysis('travel_dirs')

Analysis of field  travel_dirs
199 	 Number of unique values in  travel_dirs
0 	 Number of null values in  travel_dirs

	 Value Counts
EE      21023
WW      20301
SS      18678
NN      18589
W        7396
        ...  
ENSS        1
WWNN        1
SESS        1
SSES        1
NWSS        1
Name: travel_dirs, Length: 199, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Well, that's a mess.  I presume that in each of these codes, the first letter is the primary direction.   

In [43]:
data['travel_dirs'] = data['travel_dirs'].astype(str).apply(lambda x: x[:1])
Analysis('travel_dirs')

Analysis of field  travel_dirs
5 	 Number of unique values in  travel_dirs
0 	 Number of null values in  travel_dirs

	 Value Counts
E    41423
W    40868
S    38437
N    38206
      1252
Name: travel_dirs, dtype: int64

	 Truncated list of unique values in  travel_dirs : 
['W' 'S' 'E' 'N' ' ']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [44]:
data['travel_dirs'].replace(' ','Other', inplace=True)

In [45]:
Analysis('travel_dirs')
Make_Category('travel_dirs')

Analysis of field  travel_dirs
5 	 Number of unique values in  travel_dirs
0 	 Number of null values in  travel_dirs

	 Value Counts
E        41423
W        40868
S        38437
N        38206
Other     1252
Name: travel_dirs, dtype: int64

	 Truncated list of unique values in  travel_dirs : 
['W' 'S' 'E' 'N' 'Other']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  travel_dirs_codes
5 	 Number of unique values in  travel_dirs_codes
0 	 Number of null values in  travel_dirs_codes

	 Value Counts
0    41423
4    40868
3    38437
1    38206
2     1252
Name: travel_dirs_codes, dtype: int64

	 Truncated list of unique values in  travel_dirs_codes : 
[0, 1, 2, 3, 4]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### prior_movements
I can't find what this means, so I'm going to delete it.  

In [46]:
Analysis('prior_movements')

Analysis of field  prior_movements
1322 	 Number of unique values in  prior_movements
0 	 Number of null values in  prior_movements

	 Value Counts
BA      22845
BB      22688
B       12463
IB      11632
HB       9516
        ...  
YQN         1
JFB         1
WRR         1
WFZ         1
JBMB        1
Name: prior_movements, Length: 1322, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [47]:
data.drop('prior_movements', axis=1, inplace=True)

### crash_year

In [48]:
Analysis('crash_year')

Analysis of field  crash_year
1 	 Number of unique values in  crash_year
0 	 Number of null values in  crash_year

	 Value Counts
2019    160186
Name: crash_year, dtype: int64

	 Truncated list of unique values in  crash_year : 
[2019]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



All the same year.  Not useful.  

In [49]:
data.drop(['crash_year'], axis=1, inplace=True)

### csect

In [50]:
Analysis('csect')

Analysis of field  csect
2141 	 Number of unique values in  csect
0 	 Number of null values in  csect

	 Value Counts
          59514
450-90     2558
450-10     2059
450-15     1697
283-09     1486
          ...  
863-05        1
843-06        1
109-01        1
857-02        1
168-01        1
Name: csect, Length: 2141, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I don't know what this field represents, and there are too many unique values for it to probably be useful.

In [51]:
data.drop(['csect'], axis=1, inplace=True)

### logmile

In [52]:
Analysis('logmile')

Analysis of field  logmile
18203 	 Number of unique values in  logmile
59514 	 Number of null values in  logmile

	 Value Counts
0.000     1068
0.002       85
0.001       79
0.003       67
0.004       60
          ... 
14.419       1
12.308       1
24.155       1
34.108       1
30.935       1
Name: logmile, Length: 18202, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I'm not sure what it means, and too many unique values.  

In [53]:
data.drop(['logmile'], axis=1, inplace=True)

### lrs_id

In [54]:
Analysis('lrs_id')

Analysis of field  lrs_id
4788 	 Number of unique values in  lrs_id
0 	 Number of null values in  lrs_id

	 Value Counts
7.19E+16              10095
5.19E+16               7558
3.39E+16               6554
1.79E+16               3921
5.59E+16               3149
                      ...  
823-34-1-010              1
454-01-N-050              1
337-03-1-010              1
07120069470369A020        1
01590051650299A010        1
Name: lrs_id, Length: 4788, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Too many unique values, and I'm not sure what it is.  

In [55]:
data.drop(['lrs_id'], axis=1, inplace=True)

### lrs_logmile

In [56]:
Analysis('lrs_logmile')

Analysis of field  lrs_logmile
18093 	 Number of unique values in  lrs_logmile
1431 	 Number of null values in  lrs_logmile

	 Value Counts
0.000     2947
0.002      330
0.001      305
0.004      280
0.019      277
          ... 
19.900       1
18.124       1
17.846       1
18.410       1
4.937        1
Name: lrs_logmile, Length: 18092, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I don't think this information is useful for our purposes.

In [57]:
data.drop(['lrs_logmile'], axis=1, inplace=True)

### adt

In [58]:
Analysis('adt')

Analysis of field  adt
749 	 Number of unique values in  adt
59287 	 Number of null values in  adt

	 Value Counts
176900.0    1010
21400.0      739
11200.0      735
78600.0      725
47700.0      709
            ... 
82.0           1
84.0           1
83.0           1
52.0           1
74.0           1
Name: adt, Length: 748, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No idea what this is.

In [59]:
data.drop(['adt'], axis=1, inplace=True)

### alcohol

In [60]:
Analysis('alcohol')

Analysis of field  alcohol
2 	 Number of unique values in  alcohol
0 	 Number of null values in  alcohol

	 Value Counts
0    155530
1      4656
Name: alcohol, dtype: int64

	 Truncated list of unique values in  alcohol : 
[0 1]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Nice clean data.  Probably useful.  

### veh_type_cd1 and veh_type_cd2

| Code | Vehicle Type |
|---|---|
| A | PASSENGER CAR|	
| B | LT. TRUCK (P.U., ETC.)|	
| C | VAN|	
| D | A, B, C, OR S WITH TRAILER|	
| E | MOTORCYCLE|	
| F | PEDALCYCLE|	
| G | OFF-ROAD VEHICLE|	
| H | EMERGENCY VEHICLE IN USE|	
| I | SCHOOL BUS|	
| J | BUS W/ SEATS FOR 9-15 OCCUPANTS|	
| K | BUS W/ SEATS FOR 16 OR MORE OCC.|	
| L | SINGLE UNIT TRUCK W/ 2 AXLES|	
| M | SINGLE UNIT TRUCK W/ 3 AXLES OR MORE|	
| N | TRUCK/TRAILER|	
| P | TRUCK/TRACTOR|	
| Q | TRACTOR SEMI-TRAILER|	
| R | TRUCK DOUBLE|	
| S | SUV|	
| T | FARM EQUIPMENT|	
| V | MOTOR HOME|	
| Z | OTHER|	![image.png](attachment:image.png)

In [61]:
Analysis('veh_type_cd1')
Analysis('veh_type_cd2')

Analysis of field  veh_type_cd1
22 	 Number of unique values in  veh_type_cd1
0 	 Number of null values in  veh_type_cd1

	 Value Counts
A    70301
B    35120
S    34974
C     4903
      2999
Z     2618
Q     2314
P     1206
L     1138
E      940
D      750
M      666
I      465
F      452
H      355
N      320
K      178
G      143
J      143
V       95
T       79
R       27
Name: veh_type_cd1, dtype: int64

	 Truncated list of unique values in  veh_type_cd1 : 
['S' 'F' 'A' 'B' 'Z' ' ' 'C' 'K' 'Q' 'E' 'L' 'P' 'D' 'H' 'M' 'N' 'J' 'V'
 'R' 'G' 'T' 'I']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_type_cd2
22 	 Number of unique values in  veh_type_cd2
0 	 Number of null values in  veh_type_cd2

	 Value Counts
A    62314
S    33716
     26293
B    25906
C     4921
Q     1570
P      817
L      725
E      616
Z      508
D      484
F      444
M      417
I      404
H      308
K      256
N      208
J      117
T       73
V       53
R       21
G       15
Name: veh_type_cd2, d

Put the blanks in "Other."

In [62]:
for x in ['veh_type_cd1','veh_type_cd2']:
    data[x] = data[x].replace(' ', 'Z')
    Make_Category(x)


Analysis of field  veh_type_cd1_codes
21 	 Number of unique values in  veh_type_cd1_codes
0 	 Number of null values in  veh_type_cd1_codes

	 Value Counts
0     70301
1     35120
17    34974
20     5617
2      4903
15     2314
14     1206
11     1138
4       940
3       750
12      666
8       465
5       452
7       355
13      320
10      178
9       143
6       143
19       95
18       79
16       27
Name: veh_type_cd1_codes, dtype: int64

	 Truncated list of unique values in  veh_type_cd1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_type_cd2_codes
21 	 Number of unique values in  veh_type_cd2_codes
0 	 Number of null values in  veh_type_cd2_codes

	 Value Counts
0     62314
17    33716
20    26801
1     25906
2      4921
15     1570
14      817
11      725
4       616
3       484
5       444
12      417
8       404
7       308
10      256
13      208
9       117
18       73
19   

#### quadrant

In [63]:
Analysis('quadrant')

Analysis of field  quadrant
5 	 Number of unique values in  quadrant
0 	 Number of null values in  quadrant

	 Value Counts
      157264
3        859
1        716
2        679
4        668
Name: quadrant, dtype: int64

	 Truncated list of unique values in  quadrant : 
['  ' '2' '1' '4' '3']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No idea what this means, and almost all values are blank.  Delete!

In [64]:
data.drop(['quadrant'], axis=1, inplace=True)

### spotted_by

In [65]:
Analysis('spotted_by')

Analysis of field  spotted_by
5 	 Number of unique values in  spotted_by
0 	 Number of null values in  spotted_by

	 Value Counts
7    159366
0       492
1       242
2        52
8        34
Name: spotted_by, dtype: int64

	 Truncated list of unique values in  spotted_by : 
[7 0 8 1 2]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I cannot find what this means, and almost all of the values are the same.  Delete!

In [66]:
data.drop(['spotted_by'], axis=1, inplace=True)

### intersection_id

In [67]:
Analysis('intersection_id')

Analysis of field  intersection_id
15037 	 Number of unique values in  intersection_id
94852 	 Number of null values in  intersection_id

	 Value Counts
17LA3246@I-10                210
36US90@MARTIN LUTHER KING    199
26I-10@VETERANS#4            134
17I-10@COLLEGE               119
26US90Z@MANHATTAN            103
                            ... 
07LA4@HILL                     1
28LA92@MERMENTAU               1
17LA3246@OLD SIEGEN            1
28LA1252@TIFRERE               1
48LA6363@SATSUMA               1
Name: intersection_id, Length: 15036, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



This might be useful.  

In [68]:
data['intersection_id'].fillna('Misc', inplace=True)
intersections_misc = data['intersection_id'].value_counts() < 10
data['intersection_id'] = data['intersection_id'].apply(lambda x: 'Misc' if intersections_misc.loc[x] == True else x)
Analysis('intersection_id')

Analysis of field  intersection_id
1560 	 Number of unique values in  intersection_id
0 	 Number of null values in  intersection_id

	 Value Counts
Misc                         127328
17LA3246@I-10                   210
36US90@MARTIN LUTHER KING       199
26I-10@VETERANS#4               134
17I-10@COLLEGE                  119
                              ...  
10US171@GILLIS CUTOFF            10
52US190@AVONDALE                 10
17I-10@RIVER                     10
28US167@ROSELAWN                 10
17LA19@LOWER ZACHARY             10
Name: intersection_id, Length: 1560, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [69]:
Make_Category('intersection_id')

Analysis of field  intersection_id_codes
1560 	 Number of unique values in  intersection_id_codes
0 	 Number of null values in  intersection_id_codes

	 Value Counts
1559    127328
389        210
1112       199
601        134
328        119
         ...  
1308        10
486         10
1208        10
1470        10
169         10
Name: intersection_id_codes, Length: 1560, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### severity_cd

| Code | Meaning |
|---|---|
| A | FATAL|
| B | SEVERE|
| C | MODERATE|
| D | COMPLAINT|
| E | NO INJURY|![image.png](attachment:image.png)

In [70]:
Analysis('severity_cd')

Analysis of field  severity_cd
5 	 Number of unique values in  severity_cd
0 	 Number of null values in  severity_cd

	 Value Counts
E    114937
D     34603
C      8803
B      1162
A       681
Name: severity_cd, dtype: int64

	 Truncated list of unique values in  severity_cd : 
['E' 'D' 'C' 'B' 'A']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Check for integrity of data:  Does this correlate with 'num_tot_kil'?
It has the same number of fatal accidents (681).  Check!

In [71]:
Make_Category('severity_cd')

Analysis of field  severity_cd_codes
5 	 Number of unique values in  severity_cd_codes
0 	 Number of null values in  severity_cd_codes

	 Value Counts
4    114937
3     34603
2      8803
1      1162
0       681
Name: severity_cd_codes, dtype: int64

	 Truncated list of unique values in  severity_cd_codes : 
[0, 1, 2, 3, 4]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### city_cd

In [72]:
Analysis('city_cd')

Analysis of field  city_cd
20 	 Number of unique values in  city_cd
0 	 Number of null values in  city_cd

	 Value Counts
0     61414
1     29702
2     25271
4     13305
3      9393
18     8543
5      4503
6      3428
7      2235
9       832
8       722
14      371
16      312
11       85
10       38
13       18
12       10
15        2
17        1
44        1
Name: city_cd, dtype: int64

	 Truncated list of unique values in  city_cd : 
[ 1  2  0  5  6 18  4  3  7  9 16  8 14 10 11 13 12 15 17 44]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



I don't know whether '0' is a city; it's probably 'None."  I would guess that '1' is Baton Rouge.  Since I have no idea, I'm going to delete this field.  

In [73]:
data.drop(['city_cd'], axis=1, inplace=True)

### roadway_departure, lane_departure

In [74]:
for x in ['roadway_departure', 'lane_departure']:
    Analysis(x)

Analysis of field  roadway_departure
2 	 Number of unique values in  roadway_departure
0 	 Number of null values in  roadway_departure

	 Value Counts
No     132351
Yes     27835
Name: roadway_departure, dtype: int64

	 Truncated list of unique values in  roadway_departure : 
['No' 'Yes']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  lane_departure
2 	 Number of unique values in  lane_departure
0 	 Number of null values in  lane_departure

	 Value Counts
No     128068
Yes     32118
Name: lane_departure, dtype: int64

	 Truncated list of unique values in  lane_departure : 
['No' 'Yes']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [75]:
for x in ['roadway_departure', 'lane_departure']:
    data[x] = data[x].map({'No':0, 'Yes':1}).astype(int)
    Analysis(x)

Analysis of field  roadway_departure
2 	 Number of unique values in  roadway_departure
0 	 Number of null values in  roadway_departure

	 Value Counts
0    132351
1     27835
Name: roadway_departure, dtype: int64

	 Truncated list of unique values in  roadway_departure : 
[0 1]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  lane_departure
2 	 Number of unique values in  lane_departure
0 	 Number of null values in  lane_departure

	 Value Counts
0    128068
1     32118
Name: lane_departure, dtype: int64

	 Truncated list of unique values in  lane_departure : 
[0 1]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### road_rel_cd

| Code | Meaning |
|---|---|
| A | ON ROADWAY|
| B | SHOULDER|
| C | MEDIAN|
| D | BEYOND SHOULDER - LEFT|
| E | BEYOND SHOULDER - RIGHT|
| F | BEYOND RIGHT OF WAY|
| G | GORE|
| Y | UNKNOWN|
| Z | OTHER|![image.png](attachment:image.png)

In [76]:
Analysis('road_rel_cd')

Analysis of field  road_rel_cd
11 	 Number of unique values in  road_rel_cd
0 	 Number of null values in  road_rel_cd

	 Value Counts
A    139994
E      8270
D      4852
B      4076
Z      1309
C       685
F       495
Y       212
G       140
        114
1        39
Name: road_rel_cd, dtype: int64

	 Truncated list of unique values in  road_rel_cd : 
['A' 'D' 'B' 'F' 'E' 'Y' 'G' 'Z' ' ' 'C' '1']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Merge 'Unknown', 'Other', blank, and '1.'

In [77]:
data['road_rel_cd'].replace(['Z',' ','1'],'Y', inplace=True)
Analysis ('road_rel_cd')

Analysis of field  road_rel_cd
8 	 Number of unique values in  road_rel_cd
0 	 Number of null values in  road_rel_cd

	 Value Counts
A    139994
E      8270
D      4852
B      4076
Y      1674
C       685
F       495
G       140
Name: road_rel_cd, dtype: int64

	 Truncated list of unique values in  road_rel_cd : 
['A' 'D' 'B' 'F' 'E' 'Y' 'G' 'C']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [78]:
Make_Category('road_rel_cd')

Analysis of field  road_rel_cd_codes
8 	 Number of unique values in  road_rel_cd_codes
0 	 Number of null values in  road_rel_cd_codes

	 Value Counts
0    139994
4      8270
3      4852
1      4076
7      1674
2       685
5       495
6       140
Name: road_rel_cd_codes, dtype: int64

	 Truncated list of unique values in  road_rel_cd_codes : 
[0, 1, 2, 3, 4, 5, 6, 7]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### hwy_class

In [79]:
Analysis('hwy_class')

Analysis of field  hwy_class
19 	 Number of unique values in  hwy_class
0 	 Number of null values in  hwy_class

	 Value Counts
     59995
7    20145
5    16714
D    11878
H    11028
1    10332
6     7659
8     7436
F     6062
4     2775
B     1614
3     1420
X     1013
I     1001
S      410
G      329
C      199
2       92
A       84
Name: hwy_class, dtype: int64

	 Truncated list of unique values in  hwy_class : 
[' ' '5' '3' '7' '1' 'D' 'B' '6' 'F' 'H' 'I' '4' 'G' '8' 'S' 'X' '2' 'A'
 'C']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No idea what this means.  Drop!

In [80]:
data.drop('hwy_class', axis=1, inplace=True)

### contributing_factor

In [81]:
Analysis ('contributing_factor')

Analysis of field  contributing_factor
2 	 Number of unique values in  contributing_factor
0 	 Number of null values in  contributing_factor

	 Value Counts
R    113521
O     46665
Name: contributing_factor, dtype: int64

	 Truncated list of unique values in  contributing_factor : 
['R' 'O']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No idea what this means.  Drop!

In [82]:
data.drop('contributing_factor', axis=1, inplace=True)

### location_type

| Code | Meaning |
|---|---|
| A | MANUFACTURING OR INDUSTRIAL|
| B | BUSINESS CONTINUOUS|
| C | BUSINESS, MIXED RESIDENTIAL|
| D | RESIDENTIAL DISTRICT|
| E | RESIDENTIAL SCATTERED|
| F | SCHOOL OR PLAYGROUND|
| G | OPEN COUNTRY|
| Z | OTHER|![image.png](attachment:image.png)

In [83]:
Analysis('location_type')

Analysis of field  location_type
8 	 Number of unique values in  location_type
0 	 Number of null values in  location_type

	 Value Counts
Z    98413
A    48820
C     4646
H     4466
B     1241
D     1172
F      930
E      498
Name: location_type, dtype: int64

	 Truncated list of unique values in  location_type : 
['A' 'Z' 'H' 'C' 'F' 'D' 'B' 'E']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Funny how most of them are 'Other.'  That's either an incomplete list, a vague list, or lazy reporting.  

In [84]:
Make_Category('location_type')

Analysis of field  location_type_codes
8 	 Number of unique values in  location_type_codes
0 	 Number of null values in  location_type_codes

	 Value Counts
7    98413
0    48820
2     4646
6     4466
1     1241
3     1172
5      930
4      498
Name: location_type_codes, dtype: int64

	 Truncated list of unique values in  location_type_codes : 
[0, 1, 2, 3, 4, 5, 6, 7]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### veh_severity_cd

In [85]:
Analysis('veh_severity_cd')

Analysis of field  veh_severity_cd
5 	 Number of unique values in  veh_severity_cd
0 	 Number of null values in  veh_severity_cd

	 Value Counts
D    75347
C    44861
E    21983
B    16114
Y     1881
Name: veh_severity_cd, dtype: int64

	 Truncated list of unique values in  veh_severity_cd : 
['D' 'C' 'B' 'E' 'Y']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



No idea what this means.  Drop!

In [86]:
data.drop('veh_severity_cd', axis=1, inplace=True)

### ORIG_LATITUDE, ORIG_LONGITUDE, DOTD_LATITUDE, DOTD_LONGITUDE
If we tried to group these, we'd get something approximating 'parish_cd,' which we already have.  Drop!

In [87]:
for x in ['ORIG_LATITUDE', 'ORIG_LONGITUDE', 'DOTD_LATITUDE', 'DOTD_LONGITUDE']:
    data.drop(x, axis=1, inplace=True)

### parish_cd.1

In [88]:
Analysis('parish_cd.1')

Analysis of field  parish_cd.1
65 	 Number of unique values in  parish_cd.1
0 	 Number of null values in  parish_cd.1

	 Value Counts
17    22010
36    19942
26    16563
28    11028
9      9879
      ...  
18      117
64      106
54       92
13       72
0         7
Name: parish_cd.1, Length: 65, dtype: int64

	 Truncated list of unique values in  parish_cd.1 : 
[36 17  9 28 49 43  2  1 26 20 31 32 34 55 37 33 52  8 10 23 50 40  3 38
 53 39 35 48 29 24 61 14 59 46 44 45 54 19 60 22  6 58 62  4  5 47 16 41
 51 56 27 18 13 11 64 15 21 57 42 30 25  7 63 12  0]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [89]:
X = np.where(data['parish_cd']!=data['parish_cd.1'])
print (X)
print (len(X[0]))

(array([    24,    510,    711, ..., 159393, 159698, 159719]),)
1028


I don't know how parish_cd.1 is different from parish_cd, but it differs in about 0.7% of cases.  Perhaps it's the parish of residence of the driver of vehicle #1?  Drop?

In [90]:
data.drop('parish_cd.1', axis=1, inplace=True)

### hwy_type_cd

| Code | Meaning |
|---|---|
| A | INTERSTATE|
| B | U.S. HWY|
| C | STATE HWY|
| D | PARISH ROAD|
| E | CITY STREET|
| F | OFF ROAD/PRIVATE PROPERTY|
| G | TOLL ROAD|![image.png](attachment:image.png)

In [91]:
Analysis('hwy_type_cd')

Analysis of field  hwy_type_cd
6 	 Number of unique values in  hwy_type_cd
0 	 Number of null values in  hwy_type_cd

	 Value Counts
E    49979
C    46835
B    24022
D    19988
A    19210
G      152
Name: hwy_type_cd, dtype: int64

	 Truncated list of unique values in  hwy_type_cd : 
['E' 'D' 'C' 'G' 'B' 'A']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Nice and clean.

In [92]:
Make_Category('hwy_type_cd')

Analysis of field  hwy_type_cd_codes
6 	 Number of unique values in  hwy_type_cd_codes
0 	 Number of null values in  hwy_type_cd_codes

	 Value Counts
4    49979
2    46835
1    24022
3    19988
0    19210
5      152
Name: hwy_type_cd_codes, dtype: int64

	 Truncated list of unique values in  hwy_type_cd_codes : 
[0, 1, 2, 3, 4, 5]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### pri_hwy_num

In [93]:
Analysis('pri_hwy_num')

Analysis of field  pri_hwy_num
1006 	 Number of unique values in  pri_hwy_num
0 	 Number of null values in  pri_hwy_num

	 Value Counts
0       72669
10       9828
90       5940
190      4032
61       3669
        ...  
3275        1
1098        1
970         1
3400        1
1151        1
Name: pri_hwy_num, Length: 1006, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Let's do the same thing we did with 'intersection_id'.

In [94]:
for x in ['pri_hwy_num']:
    data[x].fillna('Misc', inplace=True)
    misc = data[x].value_counts() < 10
    data[x] = data[x].apply(lambda x: 'Misc' if misc.loc[x] == True else x)
    Analysis(x)

Analysis of field  pri_hwy_num
414 	 Number of unique values in  pri_hwy_num
0 	 Number of null values in  pri_hwy_num

	 Value Counts
0       72669
10       9828
90       5940
190      4032
61       3669
        ...  
945        10
185        10
345        10
312        10
1129       10
Name: pri_hwy_num, Length: 414, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [95]:
Make_Category('pri_hwy_num')

Analysis of field  pri_hwy_num_codes
414 	 Number of unique values in  pri_hwy_num_codes
0 	 Number of null values in  pri_hwy_num_codes

	 Value Counts
0      72669
9       9828
84      5940
148     4032
56      3669
       ...  
250       10
324       10
179       10
322       10
309       10
Name: pri_hwy_num_codes, Length: 414, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### bypass

In [96]:
Analysis('bypass')

Analysis of field  bypass
47 	 Number of unique values in  bypass
0 	 Number of null values in  bypass

	 Value Counts
      152683
B       1858
W       1549
E       1391
S        461
2        378
N        335
1        331
X        268
3        240
0        194
6        142
10       118
9         68
I         39
8         33
20        18
A         13
4         12
61         8
12         8
49         4
Y          3
b          3
45         2
79         2
71         2
C          2
42         2
5          2
`          1
22         1
51         1
90         1
O          1
73         1
88         1
38         1
30         1
96         1
55         1
80         1
Sp         1
D          1
13         1
P          1
23         1
Name: bypass, dtype: int64

	 Truncated list of unique values in  bypass : 
[' ' 'X' 'A' 'W' '0' 'B' 'S' '2' 'E' '6' '9' '3' '96' '1' '10' '4' 'N'
 '12' 'I' '61' '42' '73' '20' '49' '8' '30' '80' '45' '79' '88' 'Y' '55'
 '90' 'C' 'O' '5' '`' 'b' '22' 'P' '23' '71' '38' 

No idea what these mean.  Drop!

In [97]:
data.drop('bypass', axis=1, inplace=True)

### milepost

In [98]:
Analysis ('milepost')

Analysis of field  milepost
7172 	 Number of unique values in  milepost
0 	 Number of null values in  milepost

	 Value Counts
0.00      97863
1.00       1441
3.00        542
2.00        525
4.00        376
          ...  
4.64          1
67.54         1
226.71        1
233.90        1
906.80        1
Name: milepost, Length: 7172, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Not sure what this means.  Drop.

In [99]:
data.drop('milepost', axis=1, inplace=True)

### pri_road_name

In [100]:
Analysis('pri_road_name')

Analysis of field  pri_road_name
15724 	 Number of unique values in  pri_road_name
1 	 Number of null values in  pri_road_name

	 Value Counts
                   7298
I-10               3700
AIRLINE            2692
I10                1480
VETERANS           1371
                   ... 
HORRIDGE STREET       1
La. 15                1
TANNER                1
QUINTON               1
CC BEL                1
Name: pri_road_name, Length: 15723, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Let's do the same thing we did for intersection_id and pri_hwy_num.

In [101]:
for x in ['pri_road_name']:
    data[x].fillna('Misc', inplace=True)
    data[x].replace([' '], 'Misc', inplace=True)
    misc = data[x].value_counts() < 10
    data[x] = data[x].apply(lambda x: 'Misc' if misc.loc[x] == True else x)
    Analysis(x)

Analysis of field  pri_road_name
1843 	 Number of unique values in  pri_road_name
0 	 Number of null values in  pri_road_name

	 Value Counts
Misc          35785
I-10           3700
AIRLINE        2692
I10            1480
VETERANS       1371
              ...  
AVE F            10
LA HWY 164       10
I-12 WB          10
KENNER           10
HEMLOCK          10
Name: pri_road_name, Length: 1843, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [102]:
Make_Category('pri_road_name')

Analysis of field  pri_road_name_codes
1843 	 Number of unique values in  pri_road_name_codes
0 	 Number of null values in  pri_road_name_codes

	 Value Counts
1290    35785
675      3700
50       2692
731      1480
1757     1371
        ...  
393        10
838        10
1066       10
1138       10
0          10
Name: pri_road_name_codes, Length: 1843, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### pri_dist and pri_measure
Distance from road, and the units.  

In [103]:
Analysis('pri_dist')
Analysis('pri_measure')

Analysis of field  pri_dist
7596 	 Number of unique values in  pri_dist
0 	 Number of null values in  pri_dist

	 Value Counts
0.00       60113
100.00      6713
0.10        6136
50.00       5657
200.00      3834
           ...  
3090.00        1
62.94          1
5.51           1
113.74         1
339.26         1
Name: pri_dist, Length: 7596, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  pri_measure
3 	 Number of unique values in  pri_measure
0 	 Number of null values in  pri_measure

	 Value Counts
FT       61956
         58112
MI       40118
Name: pri_measure, dtype: int64

	 Truncated list of unique values in  pri_measure : 
['     ' 'FT   ' 'MI   ']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Let's assume that if units are not given, then it's in feet.  

Let's put these in buckets.  

In [104]:
A = data['pri_measure'].loc[data['pri_dist'] >10]
print (A)
print (A.value_counts())


2         FT   
7         FT   
11        FT   
13        FT   
15        FT   
          ...  
160176    FT   
160177    FT   
160181    FT   
160182    FT   
160184    FT   
Name: pri_measure, Length: 54073, dtype: object
FT       53192
MI         449
           432
Name: pri_measure, dtype: int64


In [105]:
A = data['pri_dist'].loc[data['pri_measure'] == 'MI   ']
print (max(A))
B = pd.qcut(A,10, duplicates='drop')
B.value_counts()

9999.0


(-2.001, 0.1]    8245
(0.21, 0.4]      4966
(1.12, 1.9]      4245
(1.9, 3.0]       4181
(0.9, 1.12]      3904
(0.1, 0.21]      3793
(0.5, 0.9]       3748
(3.0, 9999.0]    3607
(0.4, 0.5]       3429
Name: pri_dist, dtype: int64

In [106]:
A = data['pri_dist'].loc[data['pri_measure'] == 'FT   ']
B = pd.qcut(A,10,duplicates='drop')
B.value_counts()

(-0.001, 10.0]       8764
(30.0, 50.0]         8317
(268.42, 500.0]      8297
(80.0, 100.0]        7532
(183.0, 268.42]      6191
(10.0, 20.0]         5720
(20.0, 30.0]         4761
(100.0, 183.0]       4471
(500.0, 99999.99]    4094
(50.0, 80.0]         3809
Name: pri_dist, dtype: int64

- If no units given, if less than 1, list as miles; else, feet.
- Convert miles to feet.
- Put into 5 ranges
- Convert to categories.  


In [107]:
maskA = data['pri_measure']=='     '
maskB = data['pri_dist'] < 1 
maskC = data['pri_dist'] > -1 
maskD = data['pri_dist'] < 1 
maskE = data['pri_dist'] < 1 


mask = maskA & maskB 
nmask = maskA & maskC
column_name = 'pri_measure'

data.loc[mask, column_name] = 'MI   '
data.loc[nmask, column_name] = 'FT   '
Analysis('pri_measure')

Analysis of field  pri_measure
2 	 Number of unique values in  pri_measure
0 	 Number of null values in  pri_measure

	 Value Counts
FT       120068
MI        40118
Name: pri_measure, dtype: int64

	 Truncated list of unique values in  pri_measure : 
['FT   ' 'MI   ']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [108]:
mask = data['pri_measure']=='MI   '
data.loc[mask, 'pri_dist'] *= 5280
data.loc[mask, 'pri_measure'] = 'FT   '

In [109]:
data['pri_dist_bin'] = pd.qcut(data['pri_dist'].astype(float), 20, duplicates='drop')
Analysis('pri_dist_bin')

Analysis of field  pri_dist_bin
14 	 Number of unique values in  pri_dist_bin
0 	 Number of null values in  pri_dist_bin

	 Value Counts
(-10560.001, 0.0]        60118
(300.0, 528.0]           12011
(62.685, 100.0]          10025
(100.0, 200.0]            9077
(4752.0, 10032.0]         8194
(46.915, 62.685]          8009
(10032.0, 52794720.0]     7792
(2112.0, 4752.0]          7600
(20.0, 46.915]            7441
(0.0, 10.0]               6757
(528.0, 1056.0]           6303
(1056.0, 2112.0]          5954
(10.0, 20.0]              5777
(200.0, 300.0]            5128
Name: pri_dist_bin, dtype: int64

	 Truncated list of unique values in  pri_dist_bin : 
[(-10560.001, 0.0], (46.915, 62.685], (10032.0, 52794720.0], (20.0, 46.915], (300.0, 528.0], ..., (10.0, 20.0], (0.0, 10.0], (62.685, 100.0], (100.0, 200.0], (2112.0, 4752.0]]
Length: 14
Categories (14, interval[float64]): [(-10560.001, 0.0] < (0.0, 10.0] < (10.0, 20.0] < (20.0, 46.915] ... (1056.0, 2112.0] < (2112.0, 4752.0] < (4752.0, 10

In [110]:
Make_Category('pri_dist_bin')

Analysis of field  pri_dist_bin_codes
14 	 Number of unique values in  pri_dist_bin_codes
0 	 Number of null values in  pri_dist_bin_codes

	 Value Counts
0     60118
8     12011
5     10025
6      9077
12     8194
4      8009
13     7792
11     7600
3      7441
1      6757
9      6303
10     5954
2      5777
7      5128
Name: pri_dist_bin_codes, dtype: int64

	 Truncated list of unique values in  pri_dist_bin_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### pri_dir

In [111]:
Analysis ('pri_dir')

Analysis of field  pri_dir
14 	 Number of unique values in  pri_dir
3 	 Number of null values in  pri_dir

	 Value Counts
     63210
E    24686
W    24648
S    24146
N    23453
0       26
3        4
1        4
5        2
.        1
F        1
4        1
C        1
Name: pri_dir, dtype: int64

	 Truncated list of unique values in  pri_dir : 
[' ' 'E' 'W' 'N' 'S' '0' '1' 'F' '5' nan '3' '.' '4' 'C']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Since 40% of the values are blank, and 'travel_dirs' is more complete, drop this one.

In [112]:
data.drop('pri_dir', axis=1, inplace=True)

### inter_road
Intersecting Road Name

In [113]:
Analysis('inter_road')

Analysis of field  inter_road
35371 	 Number of unique values in  inter_road
10 	 Number of null values in  inter_road

	 Value Counts
                       7851
I-10                   1055
I-12                    738
US 190                  654
LA 42                   596
                       ... 
CROCHET ST                1
WOODS LOOP                1
YOKELY RD                 1
987.3  LAFITON LANE       1
S CREEK LN                1
Name: inter_road, Length: 35370, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



Let's do the same thing we did for intersection_id, pri_hwy_num, and pri_road_name.

In [114]:
for x in ['inter_road']:
    data[x].fillna('Misc', inplace=True)
    data[x].replace([' '], 'Misc', inplace=True)
    misc = data[x].value_counts() < 10
    data[x] = data[x].apply(lambda x: 'Misc' if misc.loc[x] == True else x)
    Analysis(x)

Analysis of field  inter_road
2694 	 Number of unique values in  inter_road
0 	 Number of null values in  inter_road

	 Value Counts
Misc                  70183
I-10                   1055
I-12                    738
US 190                  654
LA 42                   596
                      ...  
TATE RD                  10
CHURCHILL ST             10
W MILL ST                10
2500  SOUTHSIDE DR       10
AMERICAN WAY             10
Name: inter_road, Length: 2694, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



In [115]:
Make_Category('inter_road')

Analysis of field  inter_road_codes
2694 	 Number of unique values in  inter_road_codes
0 	 Number of null values in  inter_road_codes

	 Value Counts
1956    70183
1146     1055
1161      738
2482      654
1492      596
        ...  
1740       10
1673       10
10         10
205        10
1098       10
Name: inter_road_codes, Length: 2694, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### dr_age_1, dr_age_2

In [116]:
for x in ['dr_age_1','dr_age_2']:
    Analysis(x)

Analysis of field  dr_age_1
102 	 Number of unique values in  dr_age_1
1 	 Number of null values in  dr_age_1

	 Value Counts
200.0    18487
19.0      4814
18.0      4784
20.0      4515
21.0      4383
         ...  
5.0          2
107.0        1
98.0         1
119.0        1
4.0          1
Name: dr_age_1, Length: 101, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  dr_age_2
98 	 Number of unique values in  dr_age_2
25665 	 Number of null values in  dr_age_2

	 Value Counts
200.0    7433
25.0     3011
28.0     3005
26.0     2964
34.0     2962
         ... 
5.0         1
119.0       1
96.0        1
2.0         1
8.0         1
Name: dr_age_2, Length: 97, dtype: int64

	 Truncated list of unique values in  dr_age_2 : 
[2.0, 3.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 42.0, 43.0, 44.0, 45.0, 46.0, 49.

This mess below takes out the zeroes, runs qcut, then puts the zeroes back as a category.  The trick is that cut and qcut ignore NaN.

In [117]:
for x in ['dr_age_1','dr_age_2']:
    data[x].fillna(0, inplace=True)
    data[x] = data[x].astype(int)
    data.loc[(data[x]>100), x] = 0
    Analysis(x)
    xbin = x + '_bin'
    data[xbin] = data[x].replace(0,np.nan)    
    data[xbin] = pd.qcut(data[xbin], 5, duplicates='drop').cat.add_categories(0)
    data[xbin].fillna(0, inplace=True)
    Analysis(xbin)
    Make_Category(xbin)

Analysis of field  dr_age_1
99 	 Number of unique values in  dr_age_1
0 	 Number of null values in  dr_age_1

	 Value Counts
0     18490
19     4814
18     4784
20     4515
21     4383
      ...  
5         2
3         2
99        2
4         1
98        1
Name: dr_age_1, Length: 99, dtype: int64

	 Truncated list of unique values in  dr_age_1 : 
[ 0 20 34 27 41 40 35 26 25 33 65 18 52 29 22 28 93 23 77 32 53 76 62 56
 19 36 31 51 48 71 66 37 21 39 24 30 17 44 74 47 60 64 72 58 43 67 50 61
 42 63 68 59 45 38 49 16 55 75 70 46 69 73 86 79 57 54 80 83 81 78 88 15
 84 82 87 89 85 90 91 94 95  6 92 12  2 13  3 14  9  5 11 96 10 97  7 98
  8  4 99]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  dr_age_1_bin
6 	 Number of unique values in  dr_age_1_bin
0 	 Number of null values in  dr_age_1_bin

	 Value Counts
(22.0, 30.0]     29092
(1.999, 22.0]    28982
(40.0, 56.0]     28270
(30.0, 40.0]     27890
(56.0, 99.0]     27462
0                18490
Name: dr_age_1_bin, dtype: int64

### dr_sex_1, dr_sex_2

In [118]:
for x in ['dr_sex_1','dr_sex_2']:
    Analysis(x)
    print ('*='*30)
    data[x].fillna('U', inplace=True)
    data[x].replace(' ', 'U', inplace=True)
    Analysis(x)
    print ('*='*30)
    Make_Category(x)
    print ('*='*30)

Analysis of field  dr_sex_1
4 	 Number of unique values in  dr_sex_1
1 	 Number of null values in  dr_sex_1

	 Value Counts
M    82547
F    61549
     16089
Name: dr_sex_1, dtype: int64

	 Truncated list of unique values in  dr_sex_1 : 
['M' ' ' 'F' nan]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Analysis of field  dr_sex_1
3 	 Number of unique values in  dr_sex_1
0 	 Number of null values in  dr_sex_1

	 Value Counts
M    82547
F    61549
U    16090
Name: dr_sex_1, dtype: int64

	 Truncated list of unique values in  dr_sex_1 : 
['M' 'U' 'F']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Analysis of field  dr_sex_1_codes
3 	 Number of unique values in  dr_sex_1_codes
0 	 Number of null values in  dr_sex_1_codes

	 Value Counts
1    82547
0    61549
2    16090
Name: dr_sex_1_codes, dtype: int64

	 Truncated list of unique values in  dr_sex_1_codes : 
[0, 1, 2]
-*-*-*-*-*-

### pri_contrib_fac_cd and sec_contrib_fac_cd

| Code | Meaning |
|---|---|
| A | VIOLATIONS |
| B | MOVEMENT PRIOR TO CRASH |
| C | VISION OBSCUREMENTS |
| D | CONDITION OF DRIVER |
| E | VEHICLE CONDITIONS |
| F | ROAD SURFACE |
| G | ROADWAY CONDITION |
| H | LIGHTING |
| I | WEATHER |
| J | TRAFFIC CONTROL |
| K | KIND OF LOCATION |
| L | CONDITION OF PEDESTRIAN |
| M | PEDESTRIAN ACTIONS |

In [119]:
for x in ['pri_contrib_fac_cd', 'sec_contrib_fac_cd']:
    Analysis(x)

Analysis of field  pri_contrib_fac_cd
15 	 Number of unique values in  pri_contrib_fac_cd
0 	 Number of null values in  pri_contrib_fac_cd

	 Value Counts
A    123937
B     22839
D      6273
G      2460
E      1486
C       640
I       543
F       521
K       458
M       433
H       212
        158
J       152
L        69
1         5
Name: pri_contrib_fac_cd, dtype: int64

	 Truncated list of unique values in  pri_contrib_fac_cd : 
['A' 'G' 'B' ' ' 'D' 'J' 'E' 'K' 'M' 'I' 'H' 'F' 'C' 'L' '1']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  sec_contrib_fac_cd
15 	 Number of unique values in  sec_contrib_fac_cd
0 	 Number of null values in  sec_contrib_fac_cd

	 Value Counts
B    105427
      23758
A     19045
D      5390
K      1065
C      1061
I       964
G       925
E       744
F       743
H       551
J       234
M       215
L        63
1         1
Name: sec_contrib_fac_cd, dtype: int64

	 Truncated list of unique values in  sec_contrib_fac_cd : 
['B' 'F' ' ' 'I' 'J' 'A' '

Let's merge the 158 blanks and 5 "1"'s into a "U" category.  

In [120]:
for x in ['pri_contrib_fac_cd', 'sec_contrib_fac_cd']:
    data[x].replace([' ', '1'],'Unknown', inplace=True)
    Analysis (x)

Analysis of field  pri_contrib_fac_cd
14 	 Number of unique values in  pri_contrib_fac_cd
0 	 Number of null values in  pri_contrib_fac_cd

	 Value Counts
A          123937
B           22839
D            6273
G            2460
E            1486
C             640
I             543
F             521
K             458
M             433
H             212
Unknown       163
J             152
L              69
Name: pri_contrib_fac_cd, dtype: int64

	 Truncated list of unique values in  pri_contrib_fac_cd : 
['A' 'G' 'B' 'Unknown' 'D' 'J' 'E' 'K' 'M' 'I' 'H' 'F' 'C' 'L']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  sec_contrib_fac_cd
14 	 Number of unique values in  sec_contrib_fac_cd
0 	 Number of null values in  sec_contrib_fac_cd

	 Value Counts
B          105427
Unknown     23759
A           19045
D            5390
K            1065
C            1061
I             964
G             925
E             744
F             743
H             551
J             234
M             21

In [121]:
for x in ['pri_contrib_fac_cd', 'sec_contrib_fac_cd']:
    Make_Category(x)

Analysis of field  pri_contrib_fac_cd_codes
14 	 Number of unique values in  pri_contrib_fac_cd_codes
0 	 Number of null values in  pri_contrib_fac_cd_codes

	 Value Counts
0     123937
1      22839
3       6273
6       2460
4       1486
2        640
8        543
5        521
10       458
12       433
7        212
13       163
9        152
11        69
Name: pri_contrib_fac_cd_codes, dtype: int64

	 Truncated list of unique values in  pri_contrib_fac_cd_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  sec_contrib_fac_cd_codes
14 	 Number of unique values in  sec_contrib_fac_cd_codes
0 	 Number of null values in  sec_contrib_fac_cd_codes

	 Value Counts
1     105427
13     23759
0      19045
3       5390
10      1065
2       1061
8        964
6        925
4        744
5        743
7        551
9        234
12       215
11        63
Name: sec_contrib_fac_cd_codes, dtype: int64

	 Truncated list of unique values in  sec_c

### vision_obscure_1, vision_obscure_2

| Code | Meaning |
|---|---|
| A | RAIN, SNOW, ETC. ON WINDSHIELD |
| B | WINDSHIELD OTHERWISE OBSCURED |
| C | VISION OBSCURED BY LOAD |
| D | TREES, BUSHES, ETC. |
| E | BUILDING |
| F | EMBANKMENT |
| G | SIGN BOARDS |
| H | HILLCREST |
| I | PARKED VEHICLES |
| J | MOVING VEHICLES |
| K | BLINDED BY HEADLIGHTS |
| L | BLINDED BY SUNGLARE |
| M | DISTRACTED BY NEON LIGHTS IN FIELD OF VIEW |
| N | NO OBSCUREMENTS |
| Y | UNKNOWN |
| Z | OTHER |

In [122]:
for x in ['vision_obscure_1', 'vision_obscure_2']:
    Analysis (x)

Analysis of field  vision_obscure_1
18 	 Number of unique values in  vision_obscure_1
1 	 Number of null values in  vision_obscure_1

	 Value Counts
N    125535
Y     24280
A      3130
J      2324
Z      1890
L       814
I       706
        404
D       330
B       173
K       161
C       154
H       124
M        62
E        46
G        33
F        19
Name: vision_obscure_1, dtype: int64

	 Truncated list of unique values in  vision_obscure_1 : 
['Y' 'N' 'J' ' ' 'Z' 'A' 'I' 'B' 'D' 'C' 'L' 'K' 'F' 'H' 'M' 'E' 'G' nan]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  vision_obscure_2
18 	 Number of unique values in  vision_obscure_2
25665 	 Number of null values in  vision_obscure_2

	 Value Counts
N    124416
Y      2844
       2739
Z      1824
A      1741
J       443
I       203
L        55
M        50
B        45
K        41
D        39
H        37
C        22
E        11
G         6
F         5
Name: vision_obscure_2, dtype: int64

	 Truncated list of unique values in  vi

Let's put the blanks, the nan, the "Unknown", and the "Other" together.  

In [123]:
for x in ['vision_obscure_1', 'vision_obscure_2']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'Z'],'Y', inplace=True)
    Analysis (x)
    Make_Category(x)

Analysis of field  vision_obscure_1
15 	 Number of unique values in  vision_obscure_1
0 	 Number of null values in  vision_obscure_1

	 Value Counts
N    125535
Y     26575
A      3130
J      2324
L       814
I       706
D       330
B       173
K       161
C       154
H       124
M        62
E        46
G        33
F        19
Name: vision_obscure_1, dtype: int64

	 Truncated list of unique values in  vision_obscure_1 : 
['Y' 'N' 'J' 'A' 'I' 'B' 'D' 'C' 'L' 'K' 'F' 'H' 'M' 'E' 'G']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  vision_obscure_1_codes
15 	 Number of unique values in  vision_obscure_1_codes
0 	 Number of null values in  vision_obscure_1_codes

	 Value Counts
13    125535
14     26575
0       3130
9       2324
11       814
8        706
3        330
1        173
10       161
2        154
7        124
12        62
4         46
6         33
5         19
Name: vision_obscure_1_codes, dtype: int64

	 Truncated list of unique values in  vision_obscure_1_codes : 
[

### movement_reason_1 and movement_reason_2

| Code | Meaning |
|---|---|
| A | TO AVOID OTHER VEHICLE |
| B | TO AVOID PEDESTRIAN |
| C | TO AVOID ANIMAL |
| D | TO AVOID OTHER OBJECT |
| E | PASSING |
| F | VEHICLE OUT OF CONTROL, NOT PASSING |
| G | VEHICLE OUT OF CONTROL, PASSING |
| H | FOR TRAFFIC CONTROL |
| I | DUE TO CONGESTION |
| J | DUE TO PRIOR CRASH (COLLISION) |
| K | DUE TO DRIVER CONDITION |
| L | DUE TO DRIVER VIOLATION |
| M | DUE TO VEHICLE CONDITION (FAILURE) |
| N | DUE TO PAVEMENT CONDITION |
| O | HIGH WIND |
| P | NORMAL MOVEMENT |
| Y | UNKNOWN |
| Z | OTHER |

As usual, merge the nulls, blanks, 'unknown', and 'other.'

In [124]:
for x in ['movement_reason_1', 'movement_reason_2']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)

Analysis of field  movement_reason_1
17 	 Number of unique values in  movement_reason_1
0 	 Number of null values in  movement_reason_1

	 Value Counts
L    92770
P    25452
Y    22081
K     9478
A     2701
M     1580
C     1266
I     1044
F     1036
H      752
E      564
N      525
J      334
D      282
B      159
G      121
O       41
Name: movement_reason_1, dtype: int64

	 Truncated list of unique values in  movement_reason_1 : 
['Y' 'P' 'L' 'F' 'K' 'C' 'M' 'A' 'N' 'O' 'D' 'H' 'B' 'E' 'G' 'I' 'J']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  movement_reason_1_codes
17 	 Number of unique values in  movement_reason_1_codes
0 	 Number of null values in  movement_reason_1_codes

	 Value Counts
11    92770
15    25452
16    22081
10     9478
0      2701
12     1580
2      1266
8      1044
5      1036
7       752
4       564
13      525
9       334
3       282
1       159
6       121
14       41
Name: movement_reason_1_codes, dtype: int64

	 Truncated list of unique value

### ped_actions_1, ped_actions_2

| Code | Meaning |
|---|---|
| A | NEITHER ALCOHOL NOR DRUGS PRESENT |
| B | YES (ALCOHOL PRESENT) |
| C | YES (DRUGS PRESENT) |
| D | YES (ALCOHOL AND DRUGS PRESENT) |
| Y | UNKNOWN |
| A | CROSSING, ENTERING ROAD AT INTERSECTION |
| B | CROSSING, ENTERING ROAD NOT AT INTERSECTION |
| C | WALKING IN ROAD - WITH TRAFFIC |
| D | WALKING IN ROAD - AGAINST TRAFFIC |
| E | SLEEPING IN ROADWAY |
| F | STANDING IN ROADWAY |
| G | GETTING ON OR OFF OTHER VEHICLE |
| H | PUSHING, WORKING ON VEHICLE IN ROAD |
| I | OTHER WORKING IN ROADWAY |
| J | PLAYING IN ROADWAY |
| K | NOT IN ROADWAY |
| Y | UNKNOWN |
| Z | OTHER |

The field 'ped_actions_2' is blank; drop!


In [125]:
for x in ['ped_actions_1']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
data.drop(['ped_actions_2'], axis=1, inplace=True)

Analysis of field  ped_actions_1
12 	 Number of unique values in  ped_actions_1
0 	 Number of null values in  ped_actions_1

	 Value Counts
Y    158907
A       421
B       340
C       173
K       115
F        77
D        60
I        40
J        19
G        15
H        12
E         7
Name: ped_actions_1, dtype: int64

	 Truncated list of unique values in  ped_actions_1 : 
['Y' 'J' 'K' 'D' 'A' 'B' 'C' 'I' 'F' 'G' 'H' 'E']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  ped_actions_1_codes
12 	 Number of unique values in  ped_actions_1_codes
0 	 Number of null values in  ped_actions_1_codes

	 Value Counts
11    158907
0        421
1        340
2        173
10       115
5         77
3         60
8         40
9         19
6         15
7         12
4          7
Name: ped_actions_1_codes, dtype: int64

	 Truncated list of unique values in  ped_actions_1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### veh_lighting_1, veh_lighting_2

| Code | Meaning |
|---|---|
| A | HEADLIGHTS ON |
| B | HEADLIGHTS OFF |
| C | DAYTIME RUNNING LIGHTS |
| Y | UNKNOWN |

In [126]:
for x in ['veh_lighting_1', 'veh_lighting_2']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'D'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['ped_actions_2'], axis=1, inplace=True)

Analysis of field  veh_lighting_1
4 	 Number of unique values in  veh_lighting_1
0 	 Number of null values in  veh_lighting_1

	 Value Counts
A    50820
Y    50151
B    42766
C    16449
Name: veh_lighting_1, dtype: int64

	 Truncated list of unique values in  veh_lighting_1 : 
['Y' 'A' 'B' 'C']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_lighting_1_codes
4 	 Number of unique values in  veh_lighting_1_codes
0 	 Number of null values in  veh_lighting_1_codes

	 Value Counts
0    50820
3    50151
1    42766
2    16449
Name: veh_lighting_1_codes, dtype: int64

	 Truncated list of unique values in  veh_lighting_1_codes : 
[0, 1, 2, 3]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_lighting_2
4 	 Number of unique values in  veh_lighting_2
0 	 Number of null values in  veh_lighting_2

	 Value Counts
Y    58900
B    43117
A    41075
C    17094
Name: veh_lighting_2, dtype: int64

	 Truncated list of unique values in  veh_lighting_2 : 
['A' 'Y' 'B' 'C']
-*-

### traff_cntl_cond_1, traff_cntl_cond_2

The values in these fields suggest that it's not the 'traff_cntl_cond' in the code table, but the 'traff_cntl_cd'.

From the code table, codes for 'traff_cntl_cond_cd':

| Code | Meaning |
|---|---|
| A | CONTROLS FUNCTIONING |
| B | CONTROLS NOT FUNCTIONING |
| C | CONTROLS OBSCURED |
| D | LANE MARKING UNCLEAR OR DEFECTIVE |
| E | NO CONTROLS |
| Y | UNKNOWN |

and codes for 'traff_cntl_cd':

| Code | Meaning |
|---|---|
| A | STOP SIGN |
| B | YIELD SIGN |
| C | RED SIGNAL ON |
| D | YELLOW SIGNAL ON |
| E | GREEN SIGNAL ON |
| F | GREEN TURN ARROW ON |
| G | RIGHT TURN ON RED |
| H | LIGHT PHASE UNKNOWN |
| I | FLASHING YELLOW |
| J | FLASHING RED |
| K | OFFICER, FLAGMAN |
| L | RR CROSSING, SIGN |
| M | RR CROSSING, SIGNAL |
| N | RR CROSSING, NO CONTROL |
| O | WARNING SIGN (SCHOOL, ETC.) |
| P | SCHOOL FLASHING SPEED SIGN |
| Q | YELLOW NO PASSING LINE |
| R | WHITE DASHED LINE |
| S | YELLOW DASHED LINE |
| T | BIKE LANE |
| U | CROSSWALK |
| V | NO CONTROL |
| Y | UNKNOWN |
| Z | OTHER |![image-2.png](attachment:image-2.png)

In [127]:
for x in ['traff_cntl_cond_1', 'traff_cntl_cond_2']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'D'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['ped_actions_2'], axis=1, inplace=True)

Analysis of field  traff_cntl_cond_1
23 	 Number of unique values in  traff_cntl_cond_1
0 	 Number of null values in  traff_cntl_cond_1

	 Value Counts
R    45561
V    25543
Q    17791
A    17312
C    15088
S    10250
E     8889
Z     6707
Y     5790
B     4424
F      710
H      665
J      211
I      202
G      199
O      159
M      155
K      150
U      119
T       99
L       70
N       55
P       37
Name: traff_cntl_cond_1, dtype: int64

	 Truncated list of unique values in  traff_cntl_cond_1 : 
['A' 'Q' 'S' 'R' 'V' 'Y' 'J' 'E' 'F' 'Z' 'C' 'H' 'B' 'K' 'O' 'P' 'G' 'I'
 'N' 'U' 'T' 'M' 'L']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  traff_cntl_cond_1_codes
23 	 Number of unique values in  traff_cntl_cond_1_codes
0 	 Number of null values in  traff_cntl_cond_1_codes

	 Value Counts
16    45561
20    25543
15    17791
0     17312
2     15088
17    10250
3      8889
22     6707
21     5790
1      4424
4       710
6       665
8       211
7       202
5       199
13      15

### pri_road_dir
Don't we already have this information?

In [128]:
for x in ['pri_road_dir']:
#    data[x].fillna('Y', inplace=True)
#    data[x].replace([' ', 'D'], 'Y', inplace=True)
    Analysis (x)
#    Make_Category(x)
data.drop(['pri_road_dir'], axis=1, inplace=True)

Analysis of field  pri_road_dir
9 	 Number of unique values in  pri_road_dir
0 	 Number of null values in  pri_road_dir

	 Value Counts
      131708
W       7646
N       7053
E       6399
S       6135
SW       416
NW       343
NE       260
SE       226
Name: pri_road_dir, dtype: int64

	 Truncated list of unique values in  pri_road_dir : 
[' ' 'W' 'N' 'S' 'E' 'SE' 'NE' 'NW' 'SW']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### lighting_cd

| Code | Meaning |
|---|---|
| A | DAYLIGHT |
| B | DARK - NO STREET LIGHTS |
| C | DARK - CONTINUOUS STREET LIGHT |
| D | DARK - STREET LIGHT AT INTERSECTION ONLY |
| E | DUSK |
| F | DAWN |
| Y | UNKNOWN |
| Z | OTHER |

In [129]:
for x in ['lighting_cd']:
    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', '1', '2', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['pri_road_dir'], axis=1, inplace=True)

Analysis of field  lighting_cd
7 	 Number of unique values in  lighting_cd
0 	 Number of null values in  lighting_cd

	 Value Counts
A    116211
C     21778
B     12800
D      4171
E      2338
F      1511
Y      1377
Name: lighting_cd, dtype: int64

	 Truncated list of unique values in  lighting_cd : 
['A' 'C' 'B' 'F' 'D' 'Y' 'E']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  lighting_cd_codes
7 	 Number of unique values in  lighting_cd_codes
0 	 Number of null values in  lighting_cd_codes

	 Value Counts
0    116211
2     21778
1     12800
3      4171
4      2338
5      1511
6      1377
Name: lighting_cd_codes, dtype: int64

	 Truncated list of unique values in  lighting_cd_codes : 
[0, 1, 2, 3, 4, 5, 6]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### num_veh
This one is nice and clean.

In [130]:
for x in ['num_veh']:
#    data[x].fillna('Y', inplace=True)
#    data[x].replace([' ', '1', '2', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['pri_road_dir'], axis=1, inplace=True)

Analysis of field  num_veh
12 	 Number of unique values in  num_veh
0 	 Number of null values in  num_veh

	 Value Counts
2     124530
1      25662
3       8417
4       1253
5        219
6         66
7         25
8          9
9          2
10         1
13         1
16         1
Name: num_veh, dtype: int64

	 Truncated list of unique values in  num_veh : 
[ 2  1  3  4  6  5  7  8 16  9 10 13]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  num_veh_codes
12 	 Number of unique values in  num_veh_codes
0 	 Number of null values in  num_veh_codes

	 Value Counts
1     124530
0      25662
2       8417
3       1253
4        219
5         66
6         25
7          9
8          2
9          1
10         1
11         1
Name: num_veh_codes, dtype: int64

	 Truncated list of unique values in  num_veh_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### crash_time
We already have crash_hour.  Drop!

In [131]:
for x in ['crash_time']:
#    data[x].fillna('Y', inplace=True)
#    data[x].replace([' ', '1', '2', 'Z'], 'Y', inplace=True)
    Analysis (x)
#    Make_Category(x)
#data['crash_hour'] = data['crash_time'].dt.hour
#Analysis ('crash_hour')
data.drop(['crash_time'], axis=1, inplace=True)

Analysis of field  crash_time
1440 	 Number of unique values in  crash_time
595 	 Number of null values in  crash_time

	 Value Counts
2021-04-16 17:00:00    772
2021-04-16 16:00:00    751
2021-04-16 15:00:00    718
2021-04-16 18:00:00    691
2021-04-16 17:30:00    654
                      ... 
2021-04-16 01:41:00     10
2021-04-16 02:36:00      9
2021-04-16 04:04:00      9
2021-04-16 03:47:00      9
2021-04-16 04:18:00      8
Name: crash_time, Length: 1439, dtype: int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*



### dr_cond_cd1 and dr_cond_cd2

| Code | Meaning |
|---|---|
| A | NORMAL |
| B | INATTENTIVE |
| C | DISTRACTED |
| D | ILLNESS |
| E | FATIGUED |
| F | APPARENTLY ASLEEP/BLACKOUT |
| G | DRINKING ALCOHOL - IMPAIRED |
| H | DRINKING ALCOHOL - NOT IMPAIRED |
| I | DRUG USE - IMPAIRED |
| J | DRUG USE - NOT IMPAIRED |
| K | PHYSICAL IMPAIRMENT (EYES, EAR, LIMB) |
| Y | UNKNOWN |
| Z | OTHER |

In [132]:
for x in ['dr_cond_cd1', 'dr_cond_cd2']:
#    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['crash_time'], axis=1, inplace=True)

Analysis of field  dr_cond_cd1
12 	 Number of unique values in  dr_cond_cd1
0 	 Number of null values in  dr_cond_cd1

	 Value Counts
B    79780
A    38822
Y    25382
C     8571
G     3436
F     1439
E     1011
I      768
D      594
H      213
K      146
J       24
Name: dr_cond_cd1, dtype: int64

	 Truncated list of unique values in  dr_cond_cd1 : 
['Y' 'A' 'B' 'F' 'C' 'G' 'H' 'D' 'E' 'I' 'K' 'J']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  dr_cond_cd1_codes
12 	 Number of unique values in  dr_cond_cd1_codes
0 	 Number of null values in  dr_cond_cd1_codes

	 Value Counts
1     79780
0     38822
11    25382
2      8571
6      3436
5      1439
4      1011
8       768
3       594
7       213
10      146
9        24
Name: dr_cond_cd1_codes, dtype: int64

	 Truncated list of unique values in  dr_cond_cd1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  dr_cond_cd2
12 	 Number of unique values in  dr_cond_cd2
0 	 N

### veh_cond_cd1, veh_cond_cd2

| Code | Meaning |
|---|---|
| A | DEFECTIVE BRAKES |
| B | DEFECTIVE HEADLIGHTS |
| C | DEFECTIVE REAR LIGHTS |
| D | DEFECTIVE SIGNAL LIGHTS |
| E | ALL LIGHTS OUT |
| F | DEFECTIVE STEERING |
| G | TIRE FAILURE |
| H | WORN OR SMOOTH TIRES |
| I | ENGINE FAILURE |
| J | DEFECTIVE SUSPENSION |
| K | NO DEFECTS OBSERVED |
| Y | UNKNOWN |
| Z | OTHER |![image.png](attachment:image.png)

In [133]:
for x in ['veh_cond_cd1', 'veh_cond_cd2']:
#    data[x].fillna('Y', inplace=True)
    data[x].replace([' ', 'Z'], 'Y', inplace=True)
    Analysis (x)
    Make_Category(x)
#data.drop(['crash_time'], axis=1, inplace=True)

Analysis of field  veh_cond_cd1
12 	 Number of unique values in  veh_cond_cd1
0 	 Number of null values in  veh_cond_cd1

	 Value Counts
K    133539
Y     23900
A       884
G       722
H       486
J       184
F       162
I        90
B        82
E        60
C        55
D        22
Name: veh_cond_cd1, dtype: int64

	 Truncated list of unique values in  veh_cond_cd1 : 
['Y' 'K' 'A' 'H' 'G' 'J' 'I' 'F' 'E' 'C' 'B' 'D']
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_cond_cd1_codes
12 	 Number of unique values in  veh_cond_cd1_codes
0 	 Number of null values in  veh_cond_cd1_codes

	 Value Counts
10    133539
11     23900
0        884
6        722
7        486
9        184
5        162
8         90
1         82
4         60
2         55
3         22
Name: veh_cond_cd1_codes, dtype: int64

	 Truncated list of unique values in  veh_cond_cd1_codes : 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Analysis of field  veh_cond_cd2
12 	 Number of 

In [134]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160186 entries, 0 to 160185
Data columns (total 93 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   num_tot_kil               160186 non-null  int64         
 1   num_tot_inj               160186 non-null  int64         
 2   crash_date                160186 non-null  datetime64[ns]
 3   f_harm_ev_cd1             160186 non-null  category      
 4   m_harm_ev_cd1             160186 non-null  category      
 5   man_coll_cd               160186 non-null  category      
 6   crash_type                160186 non-null  category      
 7   surf_cond_cd              160186 non-null  category      
 8   parish_cd                 160186 non-null  int64         
 9   crash_hour                160186 non-null  object        
 10  intersection              160186 non-null  int64         
 11  invest_agency_cd          160186 non-null  object        
 12  tr

### Drop more columns